# Python script for data transformation 

## BRICARE:

BRICARE consists of 2 different types of files by year:

a. File after 2022 (2023-2024) = 79 kolom


b. File before 2022 (2019-2022) = 27 kolom


In [21]:
# Case Origin

origin_mappings = {
    "Sms": "SMS",
    "Telegram": "Telegram",
    "WebChat": "WebChat",
    "Sabrina": "Sabrina",
    "Twitter": "Twitter",
    "Facebook": "Facebook",
    "Instagram": "Instagram",
    "Pesan IB": "Pesan IB",
    "Email": "Email",
    "Video Banking": "Video Banking",
    "Media Cetak": "Media Cetak",
    "Media Online": "Media Online",
    "OJK": "OJK",
    "Lembaga Hukum Lainya": "Lembaga Hukum Lainya",
    "Media Umum Lainya": "Media Umum Lainya",
    "Phone": "Phone",
    "Surat": "Surat",
    "Walk In": "Walk In",
    "ËDC & Brilink": "EDC",
    "Brilink": "BRILink",
    "BRIMO": "BRIMO",
    "PPID": "PPID",
    "SP4N LAPOR": "SP4N LAPOR",
    "Media Konsumen": "Media Konsumen",
    "Bank Indonesia": "Bank Indonesia",
    "Ombudsman": "Ombudsman",
    "BPKN": "BPKN",
    "MMS": "MMS",
    "PKSS Oten": "PKSS Oten",
    "BRIPENS": "BRIPENS",
    "Ceria": "Ceria"
}
len(origin_mappings)

31

### File Type A


Data Extraction for File Type A must be 2 Files:


A.1 Columns (without "Details")


A.2 Details only 

Columns to be cleansed or Transform:
- All columns with values "None", "NaN, "N/A", "NULL"
- These columns must follow this datetime format: format='%Y-%m-%d %H:%M:%S' or format='%Y-%m-%d %H:%M:%S.%f' 

['Create_Date','TanggalClosed', 'tanggalTransaksi','Modified_Date','tanggalAttachmentDone','Tgl_Assigned','Tgl_Eskalasi','Tanggal_Settlement','Tgl_Foward','Tgl_In_Progress','Tgl_Returned']

- Remove all unknown characters e.g. \ufeff in column "Ticket_ID"

- Columns shoud be mapped based on their Call_Type_ID:

['Produk','Jenis_Produk','Jenis_Laporan']

- PLEASE ADD CIF

#### File A.1 Columns (without "Details"). Please use this if the file is txt

In [7]:
import pandas as pd
import re
import numpy as np

# 78 Columns
column_names = [
    "Ticket_ID", "Call_Type_ID", "Call_Type", "Create_Date", "gateway", "Jenis_Laporan", "Nama_Nasabah", 
    "No_Rekening", "Nominal", "status", "TanggalClosed", "tanggalTransaksi", "Chanel", "Fitur", "Nomor_Kartu", 
    "user_group", "assgined_to", "attachment_done", "email", "full_name", "no_telepon", "approver_login", 
    "approver_name", "SLAResolution", "submitter_login_id", "submitter_user_group", "user_login_name", 
    "Jenis_Produk", "Last_Modified_By", "Merchant_ID", "Modified_Date", "NOTAS", "Produk", "SLA_Status", "TID", 
    "tanggalAttachmentDone", "Tgl_Assigned", "Tgl_Eskalasi", "AnalisaSkils", "Attachment_", "Bank_BRI", 
    "Biaya_Admin", "Suku_Bunga", "Bunga", "Butuh_Attachment", "Cicilan", "Hasil_Kunjungan", "Log_Name", 
    "MMS_Ticket_Id", "Mass_Ticket_Upload_Flag", "Nama_Supervisor", "Nama_TL", "Nama_Wakabag", "Nasabah_Prioritas", 
    "Notify_By", "Organization", "Output_Settlement", "phone_survey", "Return_Ticket", "Settlement_By", 
    "Settlement_ID", "Settlement", "Site_User", "Status_Return", "Status_Transaksi", "Submitter_Region", 
    "Submitter_SiteGroup", "Submitter_User_group_ID", "Tanggal_Settlement", "Tgl_Foward", "Tgl_In_Progress", 
    "Tgl_Returned", "Ticket_Referensi", "Tiket_Urgency", "Tipe_Remark", "UniqueID", "users", "Usergroup_ID"
]

def parse_file(file_path):

    data = []
    date_pattern = re.compile(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}')

    with open(file_path, 'r', encoding='utf-8-sig') as file:
        for line in file:
            parts = line.strip().split(';')

            date_index = next(i for i, part in enumerate(parts) if date_pattern.match(part))

            ticket_id = parts[0] 
            call_type_id = parts[1]  
            description = ';'.join(parts[2:date_index])  
            create_date = parts[date_index]  

      
            data.append([ticket_id, call_type_id, description, create_date] + parts[date_index + 1:])


    df = pd.DataFrame(data, columns=column_names)

    df['Create_Date'] = pd.to_datetime(df['Create_Date'], errors='coerce', format='%Y-%m-%d %H:%M:%S.%f')

    return df


file_path = r"C:\Users\maste\Downloads\bricare_case_januari2023_1masking.txt"

df = parse_file(file_path)
df.replace('NULL', np.nan, inplace=True)
df.replace('None', np.nan, inplace=True)
df.replace('N/A', np.nan, inplace=True)
df.fillna('', inplace=True)
df = df.replace(['0', 0], '')


columns_to_convert = ['TanggalClosed', 'tanggalTransaksi','Modified_Date','tanggalAttachmentDone','Tgl_Assigned','Tgl_Eskalasi','Tanggal_Settlement','Tgl_Foward','Tgl_In_Progress','Tgl_Returned']
for column in columns_to_convert:
    df[column] = pd.to_datetime(df[column], format='%Y-%m-%d %H:%M:%S', errors='coerce')

    df[column] = df[column].apply(lambda x: '' if pd.isna(x) else x)
   

df['Ticket_ID'] = df['Ticket_ID'].apply(lambda x: x.replace('\ufeff', '').strip())



df.drop('Unnamed: 0', axis=1, inplace=True, errors='ignore')
startdate = pd.Timestamp(min(df['Create_Date']))
enddate = pd.Timestamp(max(df['Create_Date']))

formatted_startdate = startdate.strftime('%Y%m%d')
formatted_enddate = enddate.strftime('%Y%m%d')

filename = f"bricare_uat{formatted_startdate}_{formatted_enddate}.csv"


df.to_csv(filename, index=False)

print(f"Data saved to {filename}")

Data saved to bricare_uat20230101_20230101.csv


C:\Users\maste\AppData\Local\Temp\ipykernel_65008\3996884930.py:51: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('NULL', np.nan, inplace=True)
C:\Users\maste\AppData\Local\Temp\ipykernel_65008\3996884930.py:52: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('None', np.nan, inplace=True)
C:\Users\maste\AppData\Local\Temp\ipykernel_65008\3996884930.py:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with

#### File A.1 Columns (without "Details"). Please use this if the file is csv

In [46]:
import pandas as pd
import numpy as np

# 78 Columns
column_names = [
    "Ticket_ID", "Call_Type_ID", "Call_Type", "Create_Date", "gateway", "Jenis_Laporan", "Nama_Nasabah", 
    "No_Rekening", "Nominal", "status", "TanggalClosed", "tanggalTransaksi", "Chanel", "Fitur", "Nomor_Kartu", 
    "user_group", "assgined_to", "attachment_done", "email", "full_name", "no_telepon", "approver_login", 
    "approver_name", "SLAResolution", "submitter_login_id", "submitter_user_group", "user_login_name", 
    "Jenis_Produk", "Last_Modified_By", "Merchant_ID", "Modified_Date", "NOTAS", "Produk", "SLA_Status", "TID", 
    "tanggalAttachmentDone", "Tgl_Assigned", "Tgl_Eskalasi", "AnalisaSkils", "Attachment_", "Bank_BRI", 
    "Biaya_Admin", "Suku_Bunga", "Bunga", "Butuh_Attachment", "Cicilan", "Hasil_Kunjungan", "Log_Name", 
    "MMS_Ticket_Id", "Mass_Ticket_Upload_Flag", "Nama_Supervisor", "Nama_TL", "Nama_Wakabag", "Nasabah_Prioritas", 
    "Notify_By", "Organization", "Output_Settlement", "phone_survey", "Return_Ticket", "Settlement_By", 
    "Settlement_ID", "Settlement", "Site_User", "Status_Return", "Status_Transaksi", "Submitter_Region", 
    "Submitter_SiteGroup", "Submitter_User_group_ID", "Tanggal_Settlement", "Tgl_Foward", "Tgl_In_Progress", 
    "Tgl_Returned", "Ticket_Referensi", "Tiket_Urgency", "Tipe_Remark", "UniqueID", "users", "Usergroup_ID"
]

def parse_csv(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8-sig') as file:
        for line in file:
            parts = line.strip().split(';')
            if len(parts) > 78:
                description = ';'.join(parts[2:-75])
                new_parts = parts[:2] + [description] + parts[-75:]
                data.append(new_parts)
            else:
                data.append(parts)
    
    df = pd.DataFrame(data, columns=column_names)
    df['Create_Date'] = pd.to_datetime(df['Create_Date'], errors='coerce', format='%Y-%m-%d %H:%M:%S.%f')
    
    df.replace(['NULL', 'None', 'N/A'], np.nan, inplace=True)
    df.fillna('', inplace=True)
    df = df.replace(['0', 0], '')

    columns_to_convert = ['TanggalClosed', 'tanggalTransaksi','Modified_Date','tanggalAttachmentDone','Tgl_Assigned','Tgl_Eskalasi','Tanggal_Settlement','Tgl_Foward','Tgl_In_Progress','Tgl_Returned']
    for column in columns_to_convert:
        df[column] = pd.to_datetime(df[column], format='%Y-%m-%d %H:%M:%S', errors='coerce')
        df[column] = df[column].apply(lambda x: '' if pd.isna(x) else x)
    
    df['Ticket_ID'] = df['Ticket_ID'].apply(lambda x: x.replace('\ufeff', '').strip())

    return df
file_path = r"C:\Users\maste\Downloads\bricare_case_januari2023_1masking.csv"
df = parse_csv(file_path)

startdate = pd.Timestamp(min(df['Create_Date']))
enddate = pd.Timestamp(max(df['Create_Date']))

formatted_startdate = startdate.strftime('%Y%m%d')
formatted_enddate = enddate.strftime('%Y%m%d')

filename = f"bricare_{formatted_startdate}_{formatted_enddate}.csv"
df.to_csv(filename, index=False)

filename


C:\Users\maste\AppData\Local\Temp\ipykernel_51572\2385745217.py:35: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(['NULL', 'None', 'N/A'], np.nan, inplace=True)
C:\Users\maste\AppData\Local\Temp\ipykernel_51572\2385745217.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('', inplace=True)


'bricare_20230101_20230101.csv'

## Call Type di tes MMS

In [2]:
import pandas as pd

path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\Call Type\Data Mapping - Case_Type.csv"
df=pd.read_csv(path)
# case_types = ['8634', '8635', '8636', '8623', '8624', '8628', '8629', '8630', '8412', '8625', '8615', '8616', '8617', '8618', '8619', '8638', '8620']

case_types = ['8412']
# Filter the dataframe
df = df[df['Case Types'].isin(case_types)]
df['Segment'] = df['Segment'].replace('All', '')
df
df.to_csv(r'C:\Users\maste\Downloads\dataloader_v60.0.2\Call Type\Case_Type_MMS2.csv', index=False)

## Call Type Server

In [4]:
## use the fix document

import pandas as pd
import re

# Load the Excel file
file_path = r"C:\Users\maste\Downloads\(FINAL) SLA&OLA_NewUserGrouping_Ringkasan (4).xlsx"
df = pd.read_excel(file_path, sheet_name='FIX Upload - Update Here')

#df = df.drop(columns=['Salesforce', 'Status.1', 'Revisi Detail', 'Notes', 'User Grouping_Old', 'T4.3 - Technical Team', 'T5.1 - Third Party', 'T5.2 - Third Party', 'T5.3 - Third Party', 'Jenis Produk dan atau Layanan LKPBU', 'Kategori Permasalahan LKPBU', 'Jenis Transaksi LKPBU', 'Penyebab Pengaduan LKPBU', 'Jenis Produk LBUT', 'Sandi Jenis  Produk LBUT', 'Sebab Pengaduan LBUT', 'Sandi Penyebab LBUT'])


df=df.drop(columns=['Actual SLA\n(days) Old','Target SLA \n(days) Old', 'BRI Notes','Team Name', 'New SLA', 'Status', 'crosscek', 'GIC', 'Maker', 'Checker', 'Signer', 'Courtesy Awal', 'Kebutuhan Data'])

columns_to_replace = ['Segment', 'Product', 'Sub Product', 'Case Category']
df[columns_to_replace] = df[columns_to_replace].replace('-', '')

if 'Active' in df.columns:
    df['Active'] = df['Active'].astype(str).str.upper()


df['Detail BRICare'] = df['Detail BRICare'].fillna('-')
df

def clean_case_types(value):
    if pd.isna(value) or value == '':
        return '0'
    match = re.match(r'(\d{4})-(\d{2})', str(value))
    if match:
        year, month = match.groups()
        return f'{year}-{int(month)}'
    value = re.sub(r'\.0$', '', str(value))
    return value

df['Case Types'] = df['Case Types'].apply(clean_case_types) 
df['external_id'] = df['Case Types']

output_path = 'Cleaned4_Call_Type.xlsx'
output_path2 = 'Cleaned4_Call_Type.csv'
df.to_excel(output_path, index=False)
df.to_csv(output_path2, index=False)


In [10]:
# Check if all call types are covered

import pandas as pd

# Define the paths to your CSV files
file1_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\Call Type\calltype_psuyanto.csv"
# file2_path = r"D:\dataquality2\Combined3_Cleaned_Call_Type.csv"

file2_path = r"D:\dataquality2\Cleaned4_Call_Type.csv"

# Load the CSV files
df1 = pd.read_csv(file1_path)
df2 = pd.read_csv(file2_path)

# Extract the 'Case Types' column from each DataFrame
case_types_1 = df1['calltypeidtxt'].astype(str)
case_types_2 = df2['Case Types'].astype(str)

# Convert to sets for comparison
set_case_types_1 = set(case_types_1)
set_case_types_2 = set(case_types_2)

# Find values in file1 that are missing in file2
missing_values = set_case_types_1 - set_case_types_2

# Convert the missing values set to a DataFrame
missing_values_df = pd.DataFrame(list(missing_values), columns=['Missing Case Types'])

# Save the missing values to a new CSV file
# missing_values_df.to_csv('Missing_Case_Types.csv', index=False)
missing_values_df
# print(f"Missing values from file1 that are not in file2 have been saved to 'Missing_Case_Types.csv'")



,Missing Case Types


In [11]:
# Remove duplicates if any

# remove any duplicates

import pandas as pd

# Path to the CSV file
# path = r"D:\dataquality2\Combined3_Cleaned_Call_Type.csv"

path = r"D:\dataquality2\Cleaned4_Call_Type.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(path)

# Step 1: Check for duplicates in 'Case Types'
unique_case_types = df['Case Types'].unique()
num_unique_case_types = len(unique_case_types)

print(f"Number of unique case types: {num_unique_case_types}")

# Find and print duplicate values in the 'Case Types' column
duplicate_case_types = df['Case Types'][df['Case Types'].duplicated()].unique()
print("Duplicate case types:")
for case_type in duplicate_case_types:
    print(case_type)

# Step 2: Identify and remove rows where 'Case Types' is a duplicate and 'Active' is False

# Identify duplicate 'Case Types' values
duplicates = df[df.duplicated(subset=['Case Types'], keep=False)]

# Find rows where 'Case Types' is a duplicate and 'Active' is False
to_drop = duplicates[(duplicates['Active'] == False)]

# Drop these rows from the original DataFrame
df_cleaned = df.drop(to_drop.index)

# Optionally, save the cleaned DataFrame to a new CSV file
output_path = r"D:\dataquality2\Combined_Cleaned_Call_Type.csv"
df_cleaned.to_csv(output_path, index=False)

# Print the number of rows dropped
print(f"Number of rows dropped: {len(to_drop)}")

# Display the first few rows of the cleaned DataFrame
print(df_cleaned.head())


Number of unique case types: 520
Duplicate case types:
Number of rows dropped: 0
   No Case Types  Active               User Grouping_New        Segment  \
0   1       1000    True               Resolution Center  Individu Umum   
1   2       1002    True               Resolution Center  Individu Umum   
2   3       1003    True                 Digital Banking  Individu Umum   
3   4       1005    True  Card, Digital Lending & Assset  Individu Umum   
4   5       1006    True                Operational Risk  Individu Umum   

  Product    Sub Product                 Case Category  \
0   Loans  KTA - Digital                       Inquiry   
1   Loans  KTA - Digital  Products / Promotion Inquiry   
2   Loans  KTA - Digital                       Request   
3   Loans  KTA - Digital                       Request   
4   Loans  KTA - Digital                       Request   

                                Old Case Description  \
0  CERIA - Status Pengajuan, Aplikasi, Cara & Syarat   
1      

: 

In [ ]:
# import pandas as pd
# import numpy as np
# import re

# # Define the path to your Excel file
# path = r"C:\Users\maste\Downloads\Data Migration Monitoring.xlsx"

# # Display all columns in the DataFrame
# pd.set_option('display.max_columns', None)

# # Load the Excel file
# df = pd.read_excel(path, sheet_name='missing_calltype')

# # Replace 0 with NaN and then fill NaN with empty string
# df = df.replace(0, np.nan).fillna('')

# # Define the columns to check
# columns_to_check = [
#     'T2.1 - Supporting Unit', 'T2.2 - Supporting Unit', 'T2.3 - Supporting Unit',
#     'T3.1 - Product Owner', 'T3.2 - Product Owner', 'T3.3 - Product Owner',
#     'T4.1 - Technical Team', 'T4.2 - Technical Team'
# ]

# # Add new columns for Level 1-4 OLA
# df['Level 1 OLA'] = None
# df['Level 2 OLA'] = None
# df['Level 3 OLA'] = None
# df['Level 4 OLA'] = None

# # Function to fill in the OLA levels based on non-empty columns in order
# def fill_ola_levels(row):
#     values = []
#     for col in columns_to_check:
#         if pd.notna(row[col]):
#             values.append(row[col])
#         if len(values) == 4:
#             break
    
#     for i in range(4):
#         if i < len(values):
#             row[f'Level {i+1} OLA'] = values[i]
#         else:
#             row[f'Level {i+1} OLA'] = None
#     return row

# # Apply the function to each row
# df = df.apply(fill_ola_levels, axis=1)

# # Ensure 'Case Types' is formatted correctly and handle empty values
# def clean_case_types(value):
#     if pd.isna(value) or value == '':
#         return '0'
#     match = re.match(r'(\d{4})-(\d{2})', str(value))
#     if match:
#         year, month = match.groups()
#         return f'{year}-{int(month)}'
#     # Remove .0 from the end if present
#     value = re.sub(r'\.0$', '', str(value))
#     return value

# df['Case Types'] = df['Case Types'].apply(clean_case_types)
# df['external_id'] = df['Case Types']

# # Columns to format
# columns_to_format = [
#     'Level 1 OLA',
#     'Level 2 OLA',
#     'Level 3 OLA',
#     'Level 4 OLA',
#     'T1 - Agent CCT/Frontliner',
#     'Total SLA',
#     'New SLA'
# ]
        
# # Function to remove .0 from float values and handle empty strings
# def remove_decimal_zero(value):
#     if isinstance(value, float) and value.is_integer():
#         return int(value)
#     if value == '':
#         return np.nan
#     return value

# # Apply function to specified columns
# for column in columns_to_format:
#     df[column] = df[column].apply(remove_decimal_zero).astype('Int64')

# if 'active' in df.columns:
#     df['active'] = df['active'].astype(str).str.upper()

# # df = df.drop(columns=columns_to_check)
# df.to_csv('Cleaned3_Call_Type_missing.csv', index=False)


In [ ]:
# # Cleaning missing call type

# import pandas as pd
# import numpy as np
# import re

# # Define the path to your Excel file
# path = r"C:\Users\maste\Downloads\Data Migration Monitoring.xlsx"

# # Display all columns in the DataFrame
# pd.set_option('display.max_columns', None)

# # Load the Excel file
# df = pd.read_excel(path, sheet_name='missing_calltype')



# # Replace 0 with NaN and then fill NaN with empty string
# df = df.replace(0, np.nan).fillna('')

# # Columns to format
# columns_to_format = [
#     'No',
#     'T1 - Agent CCT/Frontliner',
#     'T2.1 - Supporting Unit',
#     'T2.2 - Supporting Unit',
#     'T2.3 - Supporting Unit',
#     'T3.1 - Product Owner',
#     'T3.2 - Product Owner',
#     'T3.3 - Product Owner',
#     'T4.1 - Technical Team',
#     'T4.2 - Technical Team',
#     'Total SLA',
#     'New SLA'
# ]

# # Ensure columns_to_format exist in df
# existing_columns_to_format = [col for col in columns_to_format if col in df.columns]

# # Function to format columns
# def format_columns(col):
#     try:
#         return str(float(col)).rstrip('.0')
#     except ValueError:
#         return col

# # Apply the formatting function to selected columns
# df[existing_columns_to_format] = df[existing_columns_to_format].applymap(format_columns)

# # Ensure the 'active' column exists before modifying
# if 'active' in df.columns:
#     df['active'] = df['active'].astype(str).str.upper()

# # Convert relevant columns to numeric for addition if they exist
# numeric_columns = [
#     'T2.2 - Supporting Unit',
#     'T2.3 - Supporting Unit',
#     'T3.1 - Product Owner',
#     'T3.2 - Product Owner',
#     'T3.3 - Product Owner',
#     'T4.1 - Technical Team',
#     'T4.2 - Technical Team'
# ]

# existing_numeric_columns = [col for col in numeric_columns if col in df.columns]

# for col in existing_numeric_columns:
#     df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

# # Combine columns with addition
# if 'T2.2 - Supporting Unit' in df.columns and 'T2.3 - Supporting Unit' in df.columns:
#     df['T2.2 - Supporting Unit'] += df['T2.3 - Supporting Unit']
#     df = df.drop(columns=['T2.3 - Supporting Unit'])

# if 'T3.1 - Product Owner' in df.columns and 'T3.2 - Product Owner' in df.columns and 'T3.3 - Product Owner' in df.columns:
#     df['T3.1 - Product Owner'] += df['T3.2 - Product Owner'] + df['T3.3 - Product Owner']
#     df = df.drop(columns=['T3.2 - Product Owner', 'T3.3 - Product Owner'])

# if 'T4.1 - Technical Team' in df.columns and 'T4.2 - Technical Team' in df.columns:
#     df['T4.1 - Technical Team'] += df['T4.2 - Technical Team']
#     df = df.drop(columns=['T4.2 - Technical Team'])

# # Format the combined columns to remove .0
# combined_columns = ['T2.2 - Supporting Unit', 'T3.1 - Product Owner', 'T4.1 - Technical Team']
# for col in combined_columns:
#     if col in df.columns:
#         df[col] = df[col].apply(format_columns)

# # Columns to replace '-' with empty string
# columns_to_replace = ['Segment', 'Product', 'Sub Product', 'Case Category']
# # Ensure columns_to_replace exist in df
# existing_columns_to_replace = [col for col in columns_to_replace if col in df.columns]

# # Replace '-' with empty string in selected columns
# df[existing_columns_to_replace] = df[existing_columns_to_replace].replace('-', '')

# # Ensure 'Case Types' is formatted correctly and handle empty values
# def clean_case_types(value):
#     if pd.isna(value) or value == '':
#         return '0'
#     match = re.match(r'(\d{4})-(\d{2})', str(value))
#     if match:
#         year, month = match.groups()
#         return f'{year}-{int(month)}'
#     # Remove .0 from the end if present
#     value = re.sub(r'\.0$', '', str(value))
#     return value

# df['Case Types'] = df['Case Types'].apply(clean_case_types)
# # Add the 'external_id' column
# df['external_id'] = df['Case Types']

# # Columns to format
# columns_to_format = [
#     'T1 - Agent CCT/Frontliner',
#     'T2.1 - Supporting Unit',
#     'T2.2 - Supporting Unit',
#     'T3.1 - Product Owner',
#     'T4.1 - Technical Team',
#     'Total SLA',
#     'New SLA'
# ]

# # Function to remove .0 from float values
# def remove_decimal_zero(value):
#     if isinstance(value, float) and value.is_integer():
#         return int(value)
#     return value

# # Apply function to specified columns
# for column in columns_to_format:
#     df[column] = df[column].apply(remove_decimal_zero)
    
# # Save the cleaned data to a CSV file
# df.to_csv('Cleaned2_Call_Type_missing.csv', index=False)




In [9]:
# import pandas as pd
# import re

# # Load the Excel file
# file_path = r"C:\Users\maste\Downloads\(FINAL) SLA&OLA_NewUserGrouping_Ringkasan (2).xlsx"
# df = pd.read_excel(file_path, sheet_name='FIX Upload - Update Here')
# # df = df.drop(columns=['Salesforce', 'Status.1', 'Revisi Detail', 'Notes', 'User Grouping_Old', 'T4.3 - Technical Team', 'T5.1 - Third Party', 'T5.2 - Third Party', 'T5.3 - Third Party', 'Jenis Produk dan atau Layanan LKPBU', 'Kategori Permasalahan LKPBU', 'Jenis Transaksi LKPBU', 'Penyebab Pengaduan LKPBU', 'Jenis Produk LBUT', 'Sandi Jenis  Produk LBUT', 'Sebab Pengaduan LBUT', 'Sandi Penyebab LBUT'])

# # Define the columns to check
# columns_to_check = [
#     'T2.1 - Supporting Unit', 'T2.2 - Supporting Unit', 'T2.3 - Supporting Unit',
#     'T3.1 - Product Owner', 'T3.2 - Product Owner', 'T3.3 - Product Owner',
#     'T4.1 - Technical Team', 'T4.2 - Technical Team'
# ]

# # Add new columns for Level 1-4 OLA
# df['Level 1 OLA'] = None
# df['Level 2 OLA'] = None
# df['Level 3 OLA'] = None
# df['Level 4 OLA'] = None

# # Function to fill in the OLA levels based on non-empty columns in order
# def fill_ola_levels(row):
#     values = []
#     for col in columns_to_check:
#         if pd.notna(row[col]):
#             values.append(row[col])
#         if len(values) == 4:
#             break
    
#     for i in range(4):
#         if i < len(values):
#             row[f'Level {i+1} OLA'] = values[i]
#         else:
#             row[f'Level {i+1} OLA'] = None
#     return row

# # Apply the function to each row
# df = df.apply(fill_ola_levels, axis=1)

# # df = df.drop(columns=columns_to_check)

# # Replace '-' with empty string in selected columns
# columns_to_replace = ['Segment', 'Product', 'Sub Product', 'Case Category']
# df[columns_to_replace] = df[columns_to_replace].replace('-', '')

# # Ensure 'Case Types' is formatted correctly and handle empty values
# def clean_case_types(value):
#     if pd.isna(value) or value == '':
#         return '0'
#     match = re.match(r'(\d{4})-(\d{2})', str(value))
#     if match:
#         year, month = match.groups()
#         return f'{year}-{int(month)}'
#     # Remove .0 from the end if present
#     value = re.sub(r'\.0$', '', str(value))
#     return value

# df['Case Types'] = df['Case Types'].apply(clean_case_types)
# df['external_id'] = df['Case Types']

# # Columns to format
# columns_to_format = [
#     'Level 1 OLA',
#     'Level 2 OLA',
#     'Level 3 OLA',
#     'Level 4 OLA',
#     'T1 - Agent CCT/Frontliner',
#     'Total SLA',
#     'New SLA'
    
# ]
		
# # Function to remove .0 from float values
# def remove_decimal_zero(value):
#     if isinstance(value, float) and value.is_integer():
#         return int(value)
#     return value

# # Apply function to specified columns
# for column in columns_to_format:
#     df[column] = df[column].apply(remove_decimal_zero).astype('Int64')


# if 'active' in df.columns:
#     df['active'] = df['active'].astype(str).str.upper()

# df['Detail BRICare'] = df['Detail BRICare'].fillna('-')
# # Save the modified dataframe to a new Excel file
# output_path = 'Cleaned3_Call_Type.xlsx'
# output_path2 = 'Cleaned3_Call_Type.csv'
# df.to_excel(output_path, index=False)
# df.to_csv(output_path2, index=False)

# # Output path of the updated file
# print(output_path)
# print(output_path2)


In [7]:
# import pandas as pd
# import numpy as np
# import re

# path = r"C:\Users\maste\Downloads\(FINAL) SLA&OLA_NewUserGrouping_Ringkasan.xlsx"
# # path = r"C:\Users\maste\Downloads\Data Migration Monitoring.xlsx"

# pd.set_option('display.max_columns', None)  
# # pd.set_option('display.max_rows', None) 
# df = pd.read_excel(path)
# df = df.drop(columns=['Salesforce', 'Status.1', 'Revisi Detail', 'Notes', 'User Grouping_Old','T4.3 - Technical Team','T5.1 - Third Party','T5.2 - Third Party','T5.3 - Third Party','Jenis Produk dan atau Layanan LKPBU','Kategori Permasalahan LKPBU','Jenis Transaksi LKPBU','Penyebab Pengaduan LKPBU','Jenis Produk LBUT','Sandi Jenis  Produk LBUT','Sebab Pengaduan LBUT','Sandi Penyebab LBUT'])

# df = df.replace(0, np.nan)
# df = df.fillna('')

# # Columns to format
# columns_to_format = [
#     'No',
#     'T1 - Agent CCT/Frontliner',
#     'T2.1 - Supporting Unit',
#     'T2.2 - Supporting Unit',
#     'T2.3 - Supporting Unit',
#     'T3.1 - Product Owner',
#     'T3.2 - Product Owner',
#     'T3.3 - Product Owner',
#     'T4.1 - Technical Team',
#     'T4.2 - Technical Team',
#     'Total SLA',
#     'New SLA'
# ]

# # Function to format columns
# def format_columns(col):
#     try:
#         # Convert to float to handle empty strings and non-numeric values
#         return str(float(col)).rstrip('.0')
#     except ValueError:
#         return col  # Return original value if conversion fails

# # Apply the formatting function to selected columns
# df[columns_to_format] = df[columns_to_format].applymap(format_columns)
# df['active'] = df['active'].astype(str).str.upper()

# # Define the columns to check
# columns_to_check = [
#     'T2.1 - Supporting Unit', 'T2.2 - Supporting Unit', 'T2.3 - Supporting Unit',
#     'T3.1 - Product Owner', 'T3.2 - Product Owner', 'T3.3 - Product Owner',
#     'T4.1 - Technical Team', 'T4.2 - Technical Team'
# ]

# # Add new columns for Level 1-4 OLA
# df['Level 1 OLA'] = None
# df['Level 2 OLA'] = None
# df['Level 3 OLA'] = None
# df['Level 4 OLA'] = None


# # Function to fill in the OLA levels based on non-empty columns in order
# def fill_ola_levels(row):
#     values = []
#     for col in columns_to_check:
#         if pd.notna(row[col]):
#             values.append(row[col])
#         if len(values) == 4:
#             break
    
#     for i in range(4):
#         if i < len(values):
#             row[f'Level {i+1} OLA'] = values[i]
#         else:
#             row[f'Level {i+1} OLA'] = None
#     return row

# # Apply the function to each row
# df = df.apply(fill_ola_levels, axis=1)

# # Apply the function to each row

# df = df.drop(columns=columns_to_check)
# # ====================================================================================================================Convert relevant columns to numeric for addition
# # df['T2.2 - Supporting Unit'] = pd.to_numeric(df['T2.2 - Supporting Unit'], errors='coerce').fillna(0)
# # df['T2.3 - Supporting Unit'] = pd.to_numeric(df['T2.3 - Supporting Unit'], errors='coerce').fillna(0)
# # df['T3.1 - Product Owner'] = pd.to_numeric(df['T3.1 - Product Owner'], errors='coerce').fillna(0)
# # df['T3.2 - Product Owner'] = pd.to_numeric(df['T3.2 - Product Owner'], errors='coerce').fillna(0)
# # df['T3.3 - Product Owner'] = pd.to_numeric(df['T3.3 - Product Owner'], errors='coerce').fillna(0)
# # df['T4.1 - Technical Team'] = pd.to_numeric(df['T4.1 - Technical Team'], errors='coerce').fillna(0)
# # df['T4.2 - Technical Team'] = pd.to_numeric(df['T4.2 - Technical Team'], errors='coerce').fillna(0)

# # # Combine columns with addition
# # df['T2.2 - Supporting Unit'] = df['T2.2 - Supporting Unit'] + df['T2.3 - Supporting Unit']
# # df = df.drop(columns=['T2.3 - Supporting Unit'])

# # df['T3.1 - Product Owner'] = df['T3.1 - Product Owner'] + df['T3.2 - Product Owner'] + df['T3.3 - Product Owner']
# # df = df.drop(columns=['T3.2 - Product Owner', 'T3.3 - Product Owner'])

# # df['T4.1 - Technical Team'] = df['T4.1 - Technical Team'] + df['T4.2 - Technical Team']
# # df = df.drop(columns=['T4.2 - Technical Team'])

# # # Format the combined columns to remove .0
# # df['T2.2 - Supporting Unit'] = df['T2.2 - Supporting Unit'].apply(format_columns)
# # df['T3.1 - Product Owner'] = df['T3.1 - Product Owner'].apply(format_columns)
# # df['T4.1 - Technical Team'] = df['T4.1 - Technical Team'].apply(format_columns)
# # =============================================================================================================================================================
# # Columns to replace '-' with empty string



# columns_to_replace = ['Segment', 'Product', 'Sub Product', 'Case Category']
# # Replace '-' with empty string in selected columns
# df[columns_to_replace] = df[columns_to_replace].replace('-', '')

# # Ensure 'Case Types' is formatted correctly and handle empty values
# def clean_case_types(value):
#     if pd.isna(value) or value == '':
#         return '0'
#     match = re.match(r'(\d{4})-(\d{2})', str(value))
#     if match:
#         year, month = match.groups()
#         return f'{year}-{int(month)}'
#     # Remove .0 from the end if present
#     value = re.sub(r'\.0$', '', str(value))
#     return value

# df['Case Types'] = df['Case Types'].apply(clean_case_types)
# df['external_id'] = df['Case Types']

# # Columns to format
# columns_to_format = [
#     'T1 - Agent CCT/Frontliner',
#     'T2.1 - Supporting Unit',
#     'T2.2 - Supporting Unit',
#     'T3.1 - Product Owner',
#     'T4.1 - Technical Team',
#     'Total SLA',
#     'New SLA'
# ]

# # Function to remove .0 from float values
# def remove_decimal_zero(value):
#     if isinstance(value, float) and value.is_integer():
#         return int(value)
#     return value

# # Apply function to specified columns
# for column in columns_to_format:
#     df[column] = df[column].apply(remove_decimal_zero)

# df.to_csv('Cleaned3_Call_Type.csv', index=False)
# # df['external_id'].unique()


C:\Users\maste\AppData\Local\Temp\ipykernel_52768\1505204889.py:41: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[columns_to_format] = df[columns_to_format].applymap(format_columns)


In [29]:
# # Cleaning missing call type

# import pandas as pd
# import numpy as np
# import re

# # Define the path to your Excel file
# path = r"C:\Users\maste\Downloads\Data Migration Monitoring.xlsx"

# # Display all columns in the DataFrame
# pd.set_option('display.max_columns', None)

# # Load the Excel file
# df = pd.read_excel(path, sheet_name='missing_calltype')



# # Replace 0 with NaN and then fill NaN with empty string
# df = df.replace(0, np.nan).fillna('')

# # Columns to format
# columns_to_format = [
#     'No',
#     'T1 - Agent CCT/Frontliner',
#     'T2.1 - Supporting Unit',
#     'T2.2 - Supporting Unit',
#     'T2.3 - Supporting Unit',
#     'T3.1 - Product Owner',
#     'T3.2 - Product Owner',
#     'T3.3 - Product Owner',
#     'T4.1 - Technical Team',
#     'T4.2 - Technical Team',
#     'Total SLA',
#     'New SLA'
# ]

# # Ensure columns_to_format exist in df
# existing_columns_to_format = [col for col in columns_to_format if col in df.columns]

# # Function to format columns
# def format_columns(col):
#     try:
#         return str(float(col)).rstrip('.0')
#     except ValueError:
#         return col

# # Apply the formatting function to selected columns
# df[existing_columns_to_format] = df[existing_columns_to_format].applymap(format_columns)

# # Ensure the 'active' column exists before modifying
# if 'active' in df.columns:
#     df['active'] = df['active'].astype(str).str.upper()

# # Convert relevant columns to numeric for addition if they exist
# numeric_columns = [
#     'T2.2 - Supporting Unit',
#     'T2.3 - Supporting Unit',
#     'T3.1 - Product Owner',
#     'T3.2 - Product Owner',
#     'T3.3 - Product Owner',
#     'T4.1 - Technical Team',
#     'T4.2 - Technical Team'
# ]

# existing_numeric_columns = [col for col in numeric_columns if col in df.columns]

# for col in existing_numeric_columns:
#     df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

# # Combine columns with addition
# if 'T2.2 - Supporting Unit' in df.columns and 'T2.3 - Supporting Unit' in df.columns:
#     df['T2.2 - Supporting Unit'] += df['T2.3 - Supporting Unit']
#     df = df.drop(columns=['T2.3 - Supporting Unit'])

# if 'T3.1 - Product Owner' in df.columns and 'T3.2 - Product Owner' in df.columns and 'T3.3 - Product Owner' in df.columns:
#     df['T3.1 - Product Owner'] += df['T3.2 - Product Owner'] + df['T3.3 - Product Owner']
#     df = df.drop(columns=['T3.2 - Product Owner', 'T3.3 - Product Owner'])

# if 'T4.1 - Technical Team' in df.columns and 'T4.2 - Technical Team' in df.columns:
#     df['T4.1 - Technical Team'] += df['T4.2 - Technical Team']
#     df = df.drop(columns=['T4.2 - Technical Team'])

# # Format the combined columns to remove .0
# combined_columns = ['T2.2 - Supporting Unit', 'T3.1 - Product Owner', 'T4.1 - Technical Team']
# for col in combined_columns:
#     if col in df.columns:
#         df[col] = df[col].apply(format_columns)

# # Columns to replace '-' with empty string
# columns_to_replace = ['Segment', 'Product', 'Sub Product', 'Case Category']
# # Ensure columns_to_replace exist in df
# existing_columns_to_replace = [col for col in columns_to_replace if col in df.columns]

# # Replace '-' with empty string in selected columns
# df[existing_columns_to_replace] = df[existing_columns_to_replace].replace('-', '')

# # Ensure 'Case Types' is formatted correctly and handle empty values
# def clean_case_types(value):
#     if pd.isna(value) or value == '':
#         return '0'
#     match = re.match(r'(\d{4})-(\d{2})', str(value))
#     if match:
#         year, month = match.groups()
#         return f'{year}-{int(month)}'
#     # Remove .0 from the end if present
#     value = re.sub(r'\.0$', '', str(value))
#     return value

# df['Case Types'] = df['Case Types'].apply(clean_case_types)
# # Add the 'external_id' column
# df['external_id'] = df['Case Types']

# # Columns to format
# columns_to_format = [
#     'T1 - Agent CCT/Frontliner',
#     'T2.1 - Supporting Unit',
#     'T2.2 - Supporting Unit',
#     'T3.1 - Product Owner',
#     'T4.1 - Technical Team',
#     'Total SLA',
#     'New SLA'
# ]

# # Function to remove .0 from float values
# def remove_decimal_zero(value):
#     if isinstance(value, float) and value.is_integer():
#         return int(value)
#     return value

# # Apply function to specified columns
# for column in columns_to_format:
#     df[column] = df[column].apply(remove_decimal_zero)
    
# # Save the cleaned data to a CSV file
# df.to_csv('Cleaned2_Call_Type_missing.csv', index=False)


C:\Users\maste\AppData\Local\Temp\ipykernel_62668\3534049870.py:48: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[existing_columns_to_format] = df[existing_columns_to_format].applymap(format_columns)


In [6]:
# check if the missing call type and Cleaned file not overlapping case types


import pandas as pd

# Define the paths to your two CSV files
file1_path = r"D:\dataquality2\Cleaned3_Call_Type_missing.csv"
file2_path = r"D:\dataquality2\Cleaned3_Call_Type.csv"

# Load the CSV files
df1 = pd.read_csv(file1_path)
df2 = pd.read_csv(file2_path)

# Extract the 'Case Types' column from each DataFrame
case_types_1 = df1['Case Types'].astype(str)
case_types_2 = df2['Case Types'].astype(str)

# Check for duplicates within each file
duplicates_1 = case_types_1[case_types_1.duplicated()]
duplicates_2 = case_types_2[case_types_2.duplicated()]

# Print duplicates within each file
if not duplicates_1.empty:
    print("Duplicates in File 1:")
    print(duplicates_1)
else:
    print("No duplicates in File 1.")

if not duplicates_2.empty:
    print("Duplicates in File 2:")
    print(duplicates_2)
else:
    print("No duplicates in File 2.")

# Check if each file has unique values in the 'Case Types' column
unique_case_types_1 = case_types_1.nunique() == len(case_types_1)
unique_case_types_2 = case_types_2.nunique() == len(case_types_2)

print(f"File 1 has all unique values: {unique_case_types_1}")
print(f"File 2 has all unique values: {unique_case_types_2}")

# Convert to sets for comparison
set_case_types_1 = set(case_types_1)
set_case_types_2 = set(case_types_2)

# Find overlapping values
overlapping_values = set_case_types_1.intersection(set_case_types_2)

if overlapping_values:
    print("There are overlapping values in 'Case Types' between the two files:")
    print(overlapping_values)
else:
    print("No overlapping values in 'Case Types' between the two files.")

# Optionally, save the results to CSV files if needed
# pd.Series(list(overlapping_values)).to_csv('Overlapping_Values.csv', index=False)

if not duplicates_1.empty:
    duplicates_1.to_csv('Duplicates_in_File1.csv', index=False)
if not duplicates_2.empty:
    duplicates_2.to_csv('Duplicates_in_File2.csv', index=False)


No duplicates in File 1.
Duplicates in File 2:
443    1201
Name: Case Types, dtype: object
File 1 has all unique values: True
File 2 has all unique values: False
No overlapping values in 'Case Types' between the two files.


In [7]:
# Combine Cleaned Call Type with Missing Call Type:

import pandas as pd

# Load the uploaded CSV files
file_1_path = r"D:\dataquality2\Cleaned3_Call_Type_missing.csv"
file_2_path = r"D:\dataquality2\Cleaned3_Call_Type.csv"

df1 = pd.read_csv(file_1_path)
df2 = pd.read_csv(file_2_path)

# Identify columns in File 2 that are not in File 1
missing_in_file_1 = set(df2.columns) - set(df1.columns)

# Ensure all columns from the larger dataframe are present in the smaller dataframe before combining
# Add missing columns to df1 with NaN values
for col in missing_in_file_1:
    df1[col] = pd.NA

# Reorder columns in df1 to match the order of columns in df2
df1 = df1[df2.columns]

# Combine the dataframes
combined_df = pd.concat([df1, df2], ignore_index=True)

# # Columns to format
# columns_to_format = [
#     'T1 - Agent CCT/Frontliner',
#     'T2.1 - Supporting Unit',
#     'T2.2 - Supporting Unit',
#     'T3.1 - Product Owner',
#     'T4.1 - Technical Team',
#     'Total SLA',
#     'New SLA'
# ]

# # Function to remove .0 from float values
# def remove_decimal_zero(value):
#     if isinstance(value, float) and value.is_integer():
#         return int(value)
#     return value

# # Apply function to specified columns
# for column in columns_to_format:
#     combined_df[column] = combined_df[column].apply(remove_decimal_zero)


# Columns to format
columns_to_format = [
    'Level 1 OLA',
    'Level 2 OLA',
    'Level 3 OLA',
    'Level 4 OLA',
    'T1 - Agent CCT/Frontliner',
    'Total SLA',
    'New SLA'
]
        
# Function to remove .0 from float values and handle empty strings
def remove_decimal_zero(value):
    if isinstance(value, float) and value.is_integer():
        return int(value)
    if value == '':
        return np.nan
    return value

# Apply function to specified columns
for column in columns_to_format:
    combined_df[column] = combined_df[column].apply(remove_decimal_zero).astype('Int64')



# Save the combined dataframe to a new CSV file
combined_file_path = 'Combined3_Cleaned_Call_Type.csv'
combined_file_path2 = 'Combined3_Cleaned_Call_Type.xlsx'
combined_df.to_csv(combined_file_path, index=False)
combined_df.to_excel(combined_file_path2, index=False)




C:\Users\maste\AppData\Local\Temp\ipykernel_1100\104105547.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([df1, df2], ignore_index=True)


In [4]:
# # Compare the call type Pak SUyanto/ bricare with master call type data


# import pandas as pd


# calltype_df = pd.read_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\Call Type\calltype_psuyanto.csv")
# # final_df = pd.read_excel(r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\Call Type\(FINAL) SLA&OLA_NewUserGrouping_Ringkasan.xlsx", sheet_name=None)
# final_call_type_df = pd.read_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\Call Type\calltype_psuyanto2.csv")

# # final_call_type_df = final_df['Full Call Type Fix']


# calltype_ids = calltype_df['calltypeidtxt'].astype(str)
# case_types = final_call_type_df['Case Types'].astype(str)


# missing_calltype_ids = calltype_ids[~calltype_ids.isin(case_types)]

# missing_calltype_ids_df = missing_calltype_ids.to_frame(name='Missing Call Type IDs')


# # missing_calltype_ids_df.to_csv(r"C:\Users\maste\Downloads\missing_calltype_ids.csv", index=False)

# # print("Missing call type IDs have been saved to missing_calltype_ids.csv")
# missing_calltype_ids_df

,Missing Call Type IDs


In [8]:
# # Add Escalation Team


# import pandas as pd

# # Load the provided CSV files
# combined_cleaned_call_type = pd.read_csv(r"D:\dataquality2\Combined_Cleaned_Call_Type.csv")
# sla_ola_new_user_grouping = pd.read_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\Call Type\(FINAL) SLA&OLA_NewUserGrouping_Ringkasan - Appendix Team Name.csv")

# # Create a mapping from the SLA & OLA dataframe
# team_mapping = sla_ola_new_user_grouping.set_index('/xc//.vs')['Full Name / Queue Name'].to_dict()

# # Adding more mappings based on the provided information
# additional_mappings = {
#     'RO BRILink': 'RO - Business Channel BRILink',
#     'RO ITE': 'RO - Information, Technology & E-Channel',
#     'BRANCH(Uker)': 'Branch - Operational Unit',
#     'CDD(CLF-1DATAM)': 'CDD - Clf-1 Credit Card Init - DATAM',
#     'CDD(CLF-1 DATAM)': 'CDD - Clf-1 Credit Card Init - DATAM',
#     'CCTFrontliner': 'CCT - Contact Center Frontliner'
# }

# # Update the original mapping with additional mappings
# team_mapping.update(additional_mappings)

# # Function to clean team names
# def clean_team_name_further(team_name):
#     if pd.isna(team_name):
#         return team_name
#     # Remove specific patterns and clean up the string
#     team_name = team_name.replace("Agent", "").replace("Include", "").strip()
#     team_name = team_name.lstrip("-").strip()
#     # Remove spaces within names for better mapping
#     team_name = team_name.replace(" ", "")
#     team_name = ' '.join(word.strip('-').strip() for word in team_name.split())
#     return team_name

# # Applying the cleaning function to the correct column 'Team Name'
# combined_cleaned_call_type['Cleaned Team Name'] = combined_cleaned_call_type['Team Name'].apply(clean_team_name_further)

# # Function to clean and translate team names with specific rules
# def clean_and_translate_team_name_with_specific_rules(team_name, mapping):
#     if pd.isna(team_name) or team_name == "" or isinstance(team_name, float):
#         return [None, None, None, None]
#     team_name_cleaned = ' '.join(word.strip('-').strip() for word in str(team_name).split())
#     # Specific rule: if the team name contains "T1-CCT/Frontliner", return None for all levels
#     if "T1-CCT/Frontliner" in team_name:
#         return [None, None, None, None]
#     # Remove "Insurance Company" and "Vendor"
#     team_name_cleaned = team_name_cleaned.replace("InsuranceCompany", "").replace("Vendor", "").strip()
#     # Translate using the mapping
#     teams = [mapping.get(t.strip(), t.strip()) for t in team_name_cleaned.split(',')]
#     return teams + [None] * (4 - len(teams))

# # Reapplying the cleaning and translation process with the specific rules
# levels = combined_cleaned_call_type['Cleaned Team Name'].apply(lambda x: clean_and_translate_team_name_with_specific_rules(x, team_mapping))

# # Ensuring the result is a DataFrame with correct None values
# level_columns = pd.DataFrame(levels.tolist(), columns=['Level 1 Escalation Team', 'Level 2 Escalation Team', 'Level 3 Escalation Team', 'Level 4 Escalation Team'], index=combined_cleaned_call_type.index)

# # Creating placeholders for OLA levels
# ola_columns = pd.DataFrame(levels.tolist(), columns=['Level 1 OLA', 'Level 2 OLA', 'Level 3 OLA', 'Level 4 OLA'], index=combined_cleaned_call_type.index)

# # Assigning the translated names to the escalation levels and placeholders to OLA levels
# combined_cleaned_call_type = pd.concat([combined_cleaned_call_type, level_columns, ola_columns], axis=1)

# # Manually correcting specific known issues
# index_to_correct = combined_cleaned_call_type['Cleaned Team Name'] == 'T1-CCT/Frontliner'
# combined_cleaned_call_type.loc[index_to_correct, ['Level 1 Escalation Team', 'Level 2 Escalation Team', 'Level 3 Escalation Team', 'Level 4 Escalation Team', 'Level 1 OLA', 'Level 2 OLA', 'Level 3 OLA', 'Level 4 OLA']] = None

# index_to_correct = combined_cleaned_call_type['Cleaned Team Name'].str.contains('CDD\(ACUOTO\),CDD\(ACUCHARGEBACK\),CDD\(CLF-1DATAM\)', na=False)
# combined_cleaned_call_type.loc[index_to_correct, ['Level 1 Escalation Team', 'Level 2 Escalation Team', 'Level 3 Escalation Team', 'Level 4 Escalation Team', 'Level 1 OLA', 'Level 2 OLA', 'Level 3 OLA', 'Level 4 OLA']] = [
#     'CDD - Auth,Chrgbck & UserCo - OTO',
#     'CDD - Auth,Chrgbck & UserCo - CHARGEBACK',
#     'CDD - Clf-1 Credit Card Init - DATAM',
#     None,
#     'CDD - Auth,Chrgbck & UserCo - OTO',
#     'CDD - Auth,Chrgbck & UserCo - CHARGEBACK',
#     'CDD - Clf-1 Credit Card Init - DATAM',
#     None
# ]

# # Keeping all specified columns
# columns_to_keep = [
#     'No', 'Case Types', 'active', 'User Grouping_New', 'Segment', 'Product', 'Sub Product', 'Case Category', 'Case Description',
#     'New Case Description', 'Required Sub-Description', 'Sub', 'Doc Intake', 'List Kebutuhan Dokumen', 'Suggestion', 'Suggestion - Breakdown',
#     'Level of Effort', 'Range of Days \n(Actual SLA)', 'Actual SLA\n(days)', 'Target SLA \n(days)', 'BRI Notes', 'Team Name', 'T1 - Agent CCT/Frontliner',
#     'T2.1 - Supporting Unit', 'T2.2 - Supporting Unit', 'T3.1 - Product Owner', 'T4.1 - Technical Team', 'Total SLA', 'New SLA', 'Status', 'GIC', 'Maker',
#     'Checker', 'Signer', 'Courtesy Awal', 'Kebutuhan Data', 'Detail BRICare', 'Unnamed: 58', 'external_id', 'Cleaned Team Name',
#     'Level 1 Escalation Team', 'Level 2 Escalation Team', 'Level 3 Escalation Team', 'Level 4 Escalation Team', 'Level 1 OLA', 'Level 2 OLA', 'Level 3 OLA', 'Level 4 OLA'
# ]

# # Selecting the specified columns
# final_dataset_with_all_columns = combined_cleaned_call_type[columns_to_keep]

# # Saving the final dataset to a new CSV file
# final_dataset_with_all_columns.to_csv('Final_Cleaned_Escalation_Teams_All_Columns.csv', index=False)



In [54]:
import pandas as pd

path=r"C:\Users\maste\Downloads\Final_Revised_Combined_Cleaned_Call_Type_with_Specific_Rules_Corrected.csv"

df = pd.read_csv(path)
columns_to_keep = [
    'No', 'Case Types', 'active', 'User Grouping_New', 'Segment', 'Product', 'Sub Product', 'Case Category', 'Case Description',
    'New Case Description', 'Required Sub-Description', 'Sub', 'Doc Intake', 'List Kebutuhan Dokumen', 'Suggestion', 'Suggestion - Breakdown',
    'Level of Effort', 'Range of Days \n(Actual SLA)', 'Actual SLA\n(days)', 'Target SLA \n(days)', 'BRI Notes', 'Team Name', 'T1 - Agent CCT/Frontliner',
    'T2.1 - Supporting Unit', 'T2.2 - Supporting Unit', 'T3.1 - Product Owner', 'T4.1 - Technical Team', 'Total SLA', 'New SLA', 'Status', 'GIC', 'Maker',
    'Checker', 'Signer', 'Courtesy Awal', 'Kebutuhan Data', 'Detail BRICare', 'Unnamed: 58', 'external_id', 'Cleaned Team Name',
    'Level 1 Escalation Team', 'Level 2 Escalation Team', 'Level 3 Escalation Team', 'Level 4 Escalation Team'
]

# Selecting the specified columns
df = df[columns_to_keep]

columns_to_process = [
    'Level 1 Escalation Team',
    'Level 2 Escalation Team',
    'Level 3 Escalation Team',
    'Level 4 Escalation Team'
]

# Replace "Vendor" and "Insurance Company" with an empty string in the specified columns
for column in columns_to_process:
    df[column] = df[column].replace(['Vendor', 'Insurance Company'], '')
    
df['Detail BRICare'] = df['Detail BRICare'].fillna('-')

df.to_csv('final_cleaned_callType.csv', index=False)
# Save to Excel
df.to_excel('final_cleaned_callType.xlsx', index=False)

#### Call type mapping for columns 'Produk', 'Jenis Produk', 'Jenis Laporan'

In [8]:
import pandas as pd


user_dataset_path = r"D:\dataquality2\bricare_uat20230101_20230101.csv"
user_df = pd.read_csv(user_dataset_path)
master_df_path = r"D:\dataquality\master_calltype.csv"
master_df = pd.read_csv(master_df_path)


master_df = master_df.rename(columns={
    'Case Types': 'Call_Type_ID', 
    'Product': 'Produk', 
    'Sub Product': 'Jenis_Produk', 
    'Case Category': 'Jenis_Laporan'
})



user_df['Call_Type_ID'] = user_df['Call_Type_ID'].astype(str)
master_df['Call_Type_ID'] = master_df['Call_Type_ID'].astype(str)


merged_df = pd.merge(user_df, master_df[['Call_Type_ID', 'Produk', 'Jenis_Produk', 'Jenis_Laporan']], on='Call_Type_ID', how='left')

user_df['Produk'] = merged_df['Produk_y']
user_df['Jenis_Produk'] = merged_df['Jenis_Produk_y']
user_df['Jenis_Laporan'] = merged_df['Jenis_Laporan_y']



user_df.to_csv(user_dataset_path, index=False)

#### File A.2 Details only. Please use this if the file is txt

In [9]:
import pandas as pd

def process_text_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Remove BOM from each line
    lines = [line.replace('\ufeff', '') for line in lines]

    entries = []
    current_entry = []
    current_ticket_id = None

    for line in lines:
        if line.startswith('TTB'):
            if current_entry:  
                entries.append((current_ticket_id, '\n'.join(current_entry)))
                current_entry = []
        
            parts = line.split(',', 3)
            if len(parts) > 3:
                current_ticket_id = parts[0]  
                current_entry.append(parts[3].strip())  
            continue
        current_entry.append(line.strip())

    if current_entry:
        entries.append((current_ticket_id, '\n'.join(current_entry)))

    return entries


def remove_bom_and_strip(df):
    return df.applymap(lambda x: x.replace('\ufeff', '').strip() if isinstance(x, str) else x)


file_path = r"C:\Users\maste\Downloads\bricare_case_januari2023_2_details.txt"
processed_data = process_text_data(file_path)


df_final = pd.DataFrame(processed_data, columns=['Ticket ID', 'Details'])

if df_final.iloc[0]['Ticket ID'] and df_final.iloc[0]['Details'].startswith(df_final.iloc[0]['Ticket ID']):
    df_final.at[0, 'Details'] = df_final.iloc[0]['Details'][len(df_final.iloc[0]['Ticket ID'])+2:]

# df_final=df_final.iloc[:10]
df_final.iloc[:10].to_csv('details_uat_20230101_20230101.csv', index=False)



#### File A.2 Details only. Please use this if the file is csv

In [49]:
import pandas as pd

def process_csv_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Remove BOM from each line
    lines = [line.replace('\ufeff', '') for line in lines]

    entries = []
    current_entry = []
    current_ticket_id = None

    for line in lines:
        if line.startswith('TTB'):
            if current_entry:
                entries.append((current_ticket_id, '\n'.join(current_entry)))
                current_entry = []
        
            parts = line.split(',', 3)
            if len(parts) > 3:
                current_ticket_id = parts[0]
                current_entry.append(parts[3].strip())
            continue
        current_entry.append(line.strip())

    if current_entry:
        entries.append((current_ticket_id, '\n'.join(current_entry)))

    return entries

file_path = r"C:\Users\maste\Downloads\bricare_case_januari2023_2_details.csv"
processed_data = process_csv_data(file_path)

df_final = pd.DataFrame(processed_data, columns=['Ticket ID', 'Details'])


if df_final.iloc[0]['Ticket ID'] and df_final.iloc[0]['Details'].startswith(df_final.iloc[0]['Ticket ID']):
    df_final.at[0, 'Details'] = df_final.iloc[0]['Details'][len(df_final.iloc[0]['Ticket ID'])+2:]


df_final = df_final.iloc[:10]
output_path = "details_20230101_20230101.csv"
df_final.to_csv(output_path, index=False)

print(f"Processed data saved to {output_path}")


Processed data saved to details_20230101_20230101.csv


#### Merge the file A.1 and file A.2

In [50]:
import pandas as pd

#just take 10 lines for an example
path=r"D:\dataquality\bricare_20230101_20230101.csv"
df=pd.read_csv(path)
df.iloc[:10].to_csv(path,index=False)

In [12]:
import pandas as pd


file_path_1 = r"D:\dataquality2\bricare_uat20230101_20230101.csv"
file_path_2 = r"D:\dataquality2\details_uat_20230101_20230101.csv"


df_tenline_bricare = pd.read_csv(file_path_1)
df_detail_bricare_10line = pd.read_csv(file_path_2)

df_detail_bricare_10line.columns = ['Ticket_ID', 'Details']

merged_df = pd.merge(df_tenline_bricare, df_detail_bricare_10line, on='Ticket_ID', how='left')


output_file_path = r"D:\dataquality2\bricare_uat_20230101_20230101.csv"


column_to_move="Details"
merged_df = merged_df[[col for col in merged_df if col != column_to_move][:3] + [column_to_move] + [col for col in merged_df if col != column_to_move][3:]] 

merged_df.to_csv(output_file_path, index=False)

### File Type B


Data Extraction for File Type B (27 columns)


Columns to be cleansed or Transform:
- All columns with values "None", "NaN, "N/A", "NULL"
- These columns must follow this datetime format: format='%Y-%m-%d %H:%M:%S' or format='%Y-%m-%d %H:%M:%S.%f' 

['TanggalClosed', 'tanggalTransaksi','Create_Date']

- Remove all unknown characters e.g. \ufeff in column "Ticket_ID" if any

- PLEASE ADD CIF



In [52]:
import pandas as pd
import numpy as np

# Define the column list
column_list = [
    "Ticket_ID",  
    "Call_Type_ID",  
    "Call_Type", 
    "Create_Date",  
    "gateway",  
    "Jenis_Laporan",  
    "Nama_Nasabah",  
    "No_Rekening", 
    "Nominal",  
    "status",  
    "TanggalClosed", 
    "tanggalTransaksi",  
    "Chanel",  
    "Fitur",  
    "Nomor_Kartu", 
    "user_group",  
    "assgined_to",  
    "attachment_done",  
    "email",  
    "full_name",  
    "no_telepon",  
    "approver_login",  
    "approver_name",  
    "SLAResolution",  
    "submitter_login_id",  
    "submitter_user_group", 
    "user_login_name"  
]


path = r"C:\Users\maste\Downloads\BRICARE_25042024 masking.csv"


data = pd.read_csv(path, delimiter=';')


data['Column1'] = data['Column1'].astype(str)
data_cleaned = data[data['Column1'].str.match(r'TTB\d+')]

data_cleaned['Column2'] = data_cleaned['Column2'].astype(str)
data_cleaned = data_cleaned[data_cleaned['Column2'].str.match(r'^\d{4}$')]

data_cleaned['Column4'] = pd.to_datetime(data_cleaned['Column4'], errors='coerce')
data_cleaned = data_cleaned.dropna(subset=['Column4'])


data_to_drop = ['Column28', 'Column29', 'Column30', 'Column31', 'Column32']
data_cleaned = data_cleaned.drop(columns=data_to_drop)


if len(data_cleaned.columns) <= len(column_list):
    data_cleaned.columns = column_list[:len(data_cleaned.columns)]


data_cleaned.replace(['NULL', 'None', 'N/A'], np.nan, inplace=True)
data_cleaned.fillna('', inplace=True)
data_cleaned = data_cleaned.replace(['0', 0], '')


columns_to_convert = ['TanggalClosed', 'tanggalTransaksi', 'Create_Date']
for column in columns_to_convert:
    data_cleaned[column] = pd.to_datetime(data_cleaned[column], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')
    data_cleaned[column] = data_cleaned[column].apply(lambda x: '' if pd.isna(x) else x)

# Just take 10 lines for an example
data_cleaned = data_cleaned.iloc[:10]


data_cleaned.drop('Unnamed: 0', axis=1, inplace=True, errors='ignore')


startdate = pd.Timestamp(min(data_cleaned['Create_Date']))
enddate = pd.Timestamp(max(data_cleaned['Create_Date']))

formatted_startdate = startdate.strftime('%Y%m%d')
formatted_enddate = enddate.strftime('%Y%m%d')

filename = f"bricare_{formatted_startdate}_{formatted_enddate}.csv"


data_cleaned.to_csv(filename, index=False)

print(f"Data saved to {filename}")


C:\Users\maste\AppData\Local\Temp\ipykernel_33500\3452762791.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['Column2'] = data_cleaned['Column2'].astype(str)


Data saved to bricare_20200101_20200101.csv


## Check the number of columns

In [55]:

path1=r"D:\dataquality\bricare_20230101_20230101.csv"
path2=r"D:\dataquality\bricare_20200101_20200101.csv"

df1=pd.read_csv(path1)
df2=pd.read_csv(path2)
print(len(df1.columns))
print(len(df2.columns))

print(df1.columns)
print(df2.columns)

79
27
Index(['Ticket_ID', 'Call_Type_ID', 'Call_Type', 'Details', 'Create_Date',
       'gateway', 'Jenis_Laporan', 'Nama_Nasabah', 'No_Rekening', 'Nominal',
       'status', 'TanggalClosed', 'tanggalTransaksi', 'Chanel', 'Fitur',
       'Nomor_Kartu', 'user_group', 'assgined_to', 'attachment_done', 'email',
       'full_name', 'no_telepon', 'approver_login', 'approver_name',
       'SLAResolution', 'submitter_login_id', 'submitter_user_group',
       'user_login_name', 'Jenis_Produk', 'Last_Modified_By', 'Merchant_ID',
       'Modified_Date', 'NOTAS', 'Produk', 'SLA_Status', 'TID',
       'tanggalAttachmentDone', 'Tgl_Assigned', 'Tgl_Eskalasi', 'AnalisaSkils',
       'Attachment_', 'Bank_BRI', 'Biaya_Admin', 'Suku_Bunga', 'Bunga',
       'Butuh_Attachment', 'Cicilan', 'Hasil_Kunjungan', 'Log_Name',
       'MMS_Ticket_Id', 'Mass_Ticket_Upload_Flag', 'Nama_Supervisor',
       'Nama_TL', 'Nama_Wakabag', 'Nasabah_Prioritas', 'Notify_By',
       'Organization', 'Output_Settlement', 'phone_

## To Compare two files

In [18]:
# Load the newly uploaded files for detailed comparison
processed_file_path_newest = r"D:\dataquality\bricare_20230101_20230101.csv"
expected_file_path_newest = r"D:\dataquality\bricare_20230101_20230101.csv"

# Read the files
processed_df_newest = pd.read_csv(processed_file_path_newest)
expected_df_newest = pd.read_csv(expected_file_path_newest)

# Check for exact match first
exact_match = processed_df_newest.equals(expected_df_newest)

# Initialize a dictionary to store detailed comparison results
comparison_details = {
    'shape_match': processed_df_newest.shape == expected_df_newest.shape,
    'columns_match': processed_df_newest.columns.equals(expected_df_newest.columns),
    'column_differences': {},
    'value_differences': {}
}

# Compare each aspect if exact match is not true
if not exact_match:
    # Check shape
    if not comparison_details['shape_match']:
        comparison_details['shape_details'] = {
            'processed_shape': processed_df_newest.shape,
            'expected_shape': expected_df_newest.shape
        }
    
    # Check columns
    if not comparison_details['columns_match']:
        comparison_details['column_details'] = {
            'processed_columns': processed_df_newest.columns.tolist(),
            'expected_columns': expected_df_newest.columns.tolist()
        }

    # Check column-by-column values
    for column in processed_df_newest.columns:
        if not processed_df_newest[column].equals(expected_df_newest[column]):
            comparison_details['column_differences'][column] = processed_df_newest[column].compare(expected_df_newest[column])

# Summarize value differences
if not comparison_details['columns_match']:
    value_differences = {}
    for column in processed_df_newest.columns:
        if not processed_df_newest[column].equals(expected_df_newest[column]):
            value_differences[column] = processed_df_newest[column].compare(expected_df_newest[column])
    comparison_details['value_differences'] = value_differences

comparison_details


{'shape_match': True,
 'columns_match': True,
 'column_differences': {},
 'value_differences': {}}

# To collect all Error logs in a path

directory_path is the error logs path as well as the location where the combined error log file will be saved


In [41]:
import os
import pandas as pd

def combine_error_logs(directory_path):
    combined_df = pd.DataFrame()

    for filename in os.listdir(directory_path):
        if filename.startswith("error"):
            file_path = os.path.join(directory_path, filename)
            df = pd.read_csv(file_path)
            combined_df = pd.concat([combined_df, df], ignore_index=True)
    
    # Normalize the 'TICKET_ID' to ensure duplicates are identified
    combined_df['TICKET_ID'] = combined_df['TICKET_ID'].str.upper()

    combined_df = combined_df.drop_duplicates(subset='TICKET_ID')

    output_path = os.path.join(directory_path, 'error_logs.csv')
    combined_df.to_csv(output_path, index=False)
    print(f"Combined error logs saved to: {output_path}")
    return output_path, combined_df

directory_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\log"
output_path, combined_df = combine_error_logs(directory_path)


Combined error logs saved to: C:\Users\maste\Downloads\dataloader_v60.0.2\log\error_logs.csv


In [3]:
import os
import pandas as pd


directory = r"C:\Users\maste\OneDrive\file\log\log"


combined_df = pd.DataFrame()


for filename in os.listdir(directory):
    if 'error' in filename:
        file_path = os.path.join(directory, filename)
        try:
            df = pd.read_csv(file_path)
            combined_df = pd.concat([combined_df, df], ignore_index=True)
        except Exception as e:
            print(f"Could not read {filename}: {e}")


combined_file_path = os.path.join(directory, 'combined_errors.csv')
combined_df.to_csv(combined_file_path, index=False)

print(f"Combined file saved to {combined_file_path}")


Combined file saved to C:\Users\maste\OneDrive\file\log\log\combined_errors.csv


# To cleanse user data

In [49]:
import pandas as pd
from io import StringIO
import html


# Function to decode HTML entities in a DataFrame
def decode_html_entities(df):
    df_decoded = df.applymap(lambda x: html.unescape(x) if isinstance(x, str) else x)
    return df_decoded


file_path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\USER.txt" 

# Read the raw file content
with open(file_path, 'r') as file:
    raw_data = file.readlines()
    
# Split headers and data    
header = raw_data[0].replace("&#124;", "|").strip()
data = [line.replace("&#124;", "|").strip() for line in raw_data[1:]]

# Read the cleaned data into a pandas DataFrame
df_cleaned = pd.DataFrame([line.split('|') for line in data], columns=header.split('|'))

# Decode HTML entities
df_cleaned = decode_html_entities(df_cleaned)
df_cleaned

# Remove the quotes in Dataframe
def remove_quotes(df):
    df.columns = df.columns.str.replace('"', '')
    df = df.apply(lambda col: col.str.replace('"', '', regex=True))
    return df

df_cleaned = remove_quotes(df_cleaned)
df_cleaned
df_cleaned.to_csv("user_data.csv", index=False)

C:\Users\maste\AppData\Local\Temp\ipykernel_43760\4048578058.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_decoded = df.applymap(lambda x: html.unescape(x) if isinstance(x, str) else x)


# Working on the data

In [3]:
import pandas as pd

path = r"D:\Salesforce\archive\dataquality\gx\BRICARE_25042024.csv"

# Option 1: Check for quoted fields or use different delimiter
try:
    df = pd.read_csv(path, sep=',', on_bad_lines='skip') 
except pd.errors.ParserError as e:
    print(f"Error parsing file: {e}")


C:\Users\maste\AppData\Local\Temp\ipykernel_15156\3036651791.py:7: DtypeWarning: Columns (1,8,17,21,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep=',', on_bad_lines='skip')


In [4]:
import pandas as pd
import numpy as np

# Define the column list
column_list = [
    "Ticket_ID",  
    "Call_Type_ID",  
    "Call_Type", 
    "Create_Date",  
    "gateway",  
    "Jenis_Laporan",  
    "Nama_Nasabah",  
    "No_Rekening", 
    "Nominal",  
    "status",  
    "TanggalClosed", 
    "tanggalTransaksi",  
    "Chanel",  
    "Fitur",  
    "Nomor_Kartu", 
    "user_group",  
    "assgined_to",  
    "attachment_done",  
    "email",  
    "full_name",  
    "no_telepon",  
    "approver_login",  
    "approver_name",  
    "SLAResolution",  
    "submitter_login_id",  
    "submitter_user_group", 
    "user_login_name"
]

path = r"D:\Salesforce\archive\dataquality\gx\BRICARE_25042024.csv"

# Initialize lists to store data and problematic lines
data = []
problematic_lines = []

def parse_line(line):
    parts = line.split(',')
    try:
        # Identifying Ticket_ID
        ticket_id_index = next(i for i, part in enumerate(parts) if part.startswith('TTB'))
        ticket_id = parts[ticket_id_index]
        
        # Identifying Call_Type_ID
        call_type_id_index = ticket_id_index + 1
        call_type_id = parts[call_type_id_index]
        
        # Identifying Create_Date (the part that looks like a datetime string)
        create_date_index = next(i for i, part in enumerate(parts) if '-' in part and ':' in part)
        create_date = parts[create_date_index]
        
        # Call_Type is everything between Call_Type_ID and Create_Date
        call_type = ' '.join(parts[call_type_id_index + 1:create_date_index])
        
        # The rest of the fields in order
        rest = parts[create_date_index + 1:]
        
        # Combine into a single list in the correct order
        structured_line = [ticket_id, call_type_id, call_type, create_date] + rest
        
        # If the length is correct, return it
        if len(structured_line) == len(column_list):
            return structured_line
        # If there are extra fields, handle them (for example, by merging or ignoring)
        elif len(structured_line) > len(column_list):
            return structured_line[:len(column_list)]
        else:
            return None
    except StopIteration:
        # If any part of the parsing fails, consider the line problematic
        return None

# Read the file line by line and parse it
with open(path, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    i = 0
    while i < len(lines):
        parsed_line = parse_line(lines[i])
        if parsed_line:
            data.append(parsed_line)
            i += 1
        else:
            # Check if merging the next line solves the issue
            if i + 1 < len(lines):
                merged_line = lines[i].strip() + ' ' + lines[i + 1].strip()
                parsed_line = parse_line(merged_line)
                if parsed_line:
                    data.append(parsed_line)
                    i += 2
                else:
                    problematic_lines.append((i, lines[i]))
                    i += 1
            else:
                problematic_lines.append((i, lines[i]))
                i += 1

# Convert the collected data into a DataFrame
df = pd.DataFrame(data, columns=column_list)


df['Ticket_ID'] = df['Ticket_ID'].astype(str)
df = df[df['Ticket_ID'].str.match(r'TTB\d+')]

df['Call_Type_ID'] = df['Call_Type_ID'].astype(str)
df = df[df['Call_Type_ID'].str.match(r'^\d{4}$')]

df['Create_Date'] = pd.to_datetime(df['Create_Date'], errors='coerce')
df = df.dropna(subset=['Create_Date'])

df.replace(['NULL', 'None', 'N/A'], np.nan, inplace=True)
df.fillna('', inplace=True)
df = df.replace(['0', 0], '')

columns_to_convert = ['TanggalClosed', 'tanggalTransaksi', 'Create_Date']
for column in columns_to_convert:
    df[column] = pd.to_datetime(df[column], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')
    df[column] = df[column].apply(lambda x: '' if pd.isna(x) else x)


df.drop('Unnamed: 0', axis=1, inplace=True, errors='ignore')


startdate = pd.Timestamp(min(df['Create_Date']))
enddate = pd.Timestamp(max(df['Create_Date']))

formatted_startdate = startdate.strftime('%Y%m%d')
formatted_enddate = enddate.strftime('%Y%m%d')

filename = f"bricaredatareal_{formatted_startdate}_{formatted_enddate}.csv"


df.to_csv(filename, index=False)

print(f"Data saved to {filename}")


problematic_df = pd.DataFrame(problematic_lines, columns=['Line_Number', 'Data'])


problematic_filename = f"bricaredatareal_problematic_{formatted_startdate}_{formatted_enddate}.csv"
problematic_df.to_csv(problematic_filename, index=False)

print(f"Problematic data saved to {problematic_filename}")


print(f"Number of rows in dataframe: {len(df)}")
print(f"Number of problematic lines: {len(problematic_df)}")


Data saved to bricaredatareal_20200101_20200201.csv
Problematic data saved to bricaredatareal_problematic_20200101_20200201.csv
Number of rows in dataframe: 463581
Number of problematic lines: 881


# Create masked Data for UAT


To Mask:

Nama_Nasabah
No_Rekening
full_name
no_telepon
approver_name = beda dengan Nama_Nasabah
user_login_name = beda 
Log_Name
Nama_Supervisor
Nama_TL
Nama_Wakabag
Remark

Kolom Detail:

Kode Cabang          : 0307 4 digits
No Kartu             : 5221843130736932 16 digits
No Rekening          : 030701098507501 15 digits
Nama                 : SITI SHOLEHA
No ID                : 3316022012770004

In [13]:
import pandas as pd
from faker import Faker
import random 

fake = Faker('id_ID')
pd.set_option('display.max_columns', None)


path = r"D:\dataquality2\bricare_uat_20230101_20230101.csv"
df = pd.read_csv(path)

def mask_data(df):
    df['Nama_Nasabah'] = df['Nama_Nasabah'].apply(lambda x: fake.name())
    df['No_Rekening'] = df['No_Rekening'].apply(lambda x: fake.bban())
    df['full_name'] = df['full_name'].apply(lambda x: fake.name())
    df['no_telepon'] = df['no_telepon'].apply(lambda x: fake.phone_number())
    df['approver_name'] = df['approver_name'].apply(lambda x: fake.name())
    df['user_login_name'] = df['user_login_name'].apply(lambda x: fake.user_name())
    df['Log_Name'] = df['Log_Name'].apply(lambda x: fake.user_name())
    df['Nama_Supervisor'] = df['Nama_Supervisor'].apply(lambda x: fake.name())
    df['Nama_TL'] = df['Nama_TL'].apply(lambda x: fake.name())
    df['Nama_Wakabag'] = df['Nama_Wakabag'].apply(lambda x: fake.name())
    return df


df_masked = mask_data(df)

def generate_nik():
    return f'{fake.random_number(digits=16)}'

def mask_detail(detail):
    if isinstance(detail, float) and pd.isna(detail):
        return detail
    lines = str(detail).split('\n')
    masked_lines = []
    for line in lines:
        if 'Kode Cabang' in line:
            line = f'Kode Cabang          : {fake.random_number(digits=4)}'
        elif 'No Kartu' in line:
            line = f'No Kartu             : {fake.credit_card_number()}'
        elif 'No Rekening' in line:
            line = f'No Rekening          : {fake.bban()}'
        elif 'Nama' in line:
            line = f'Nama                 : {fake.name()}'
        elif 'No ID' in line:
            line = f'No ID                : {generate_nik()}'
        masked_lines.append(line)
    return '\n'.join(masked_lines)

# df_masked = df_masked.iloc[:10]


# Ensure 'Details' column is treated as string and apply mask_detail function
df_masked['Details'] = df_masked['Details'].astype(str).apply(mask_detail)



# gateway_options = [
#     'Email', 'Phone', 'Instagram', 'Walk-In', 'MMS', 
#     'Twitter', 'Facebook', 'BRImo', 'BRILink', 'Sabrina', 'Ceria'
# ]

#UAT
gateway_options = [
    'Email', 'Phone', 'Web', 'Facebook', 'Twitter'
]
# Assign random values to the 'gateway' column
df_masked['gateway'] = df_masked['gateway'].apply(lambda x: random.choice(gateway_options))




# df_masked = df_masked.iloc[:10]
df_masked
output_path = r"D:\dataquality2\bricare_uat_20230101_20230101_masking.csv"
df_masked.to_csv(output_path, index=False)


In [19]:
import pandas as pd

path=r"bricare_uat_20230101_20230101_masking.csv"
df=pd.read_csv(path)
df['status']='New'
df.to_csv(r"bricare_uat_20230101_20230101_masking_new.csv", index=False)

# To create data to test in SIT


In [94]:
import pandas as pd


path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\SIT\dummy_data_case1100000.csv"

df=pd.read_csv(path) 
df=df.iloc[:60000]
# df['User*']='00GMR0000000dCf2AI'
df['User*']='00GMR0000000dEH2AY'
df['Status']='Escalated'
df['Status'] = df['Status'].replace('New', 'Escalated')
df['Case Origin']='Email'
df['Case Type']='a1BMR00000010h72AA'
df['Account']='001MR000004C7l2YAC'

df.to_csv('dummy_data_uat60k.csv', index=False)

# df['Merchant ID']
# df['TID']
# df['User*']

# To create data to test in UAT

In [92]:
import pandas as pd


path=r"D:\dataquality\bricare_20230101_20230101.csv"

df=pd.read_csv(path) 
df=df.iloc[:60000]
df.to_csv('dummy_data_case_uat60k.csv', index=False)

In [10]:
import pandas as pd


file_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\data check\CHM\CHM\Part2\bricare_get_cleansing_77_kolom(1).csv"
df = pd.read_csv(file_path, delimiter=';')


output_file_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\data check\bricare_get_cleansing_77_kolom_fixed_return.csv"
df=df.iloc[:1]
df['status']='Return'
df.to_csv(output_file_path, index=False)

# output_file_path

# Feature ID        

In [54]:
import pandas as pd

# Load the CSV files
ref_fiturid_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\Call Type\ref_fiturid.csv"
id_calltype_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\extract_calltype_uat.csv"

ref_fiturid = pd.read_csv(ref_fiturid_path)
id_calltype = pd.read_csv(id_calltype_path)

# Convert calltype and NAME columns to string type
ref_fiturid['calltype'] = ref_fiturid['calltype'].astype(str)
id_calltype['NAME'] = id_calltype['NAME'].astype(str)

# Merge the dataframes to add the Call_Type_ID column based on the calltype column
ref_fiturid = ref_fiturid.merge(id_calltype[['NAME', 'ID']], left_on='calltype', right_on='NAME', how='left')

# Rename the ID column to Call_Type_ID
ref_fiturid.rename(columns={'ID': 'Call_Type_ID'}, inplace=True)

# Drop the unnecessary NAME column from the merge
ref_fiturid.drop(columns=['NAME'], inplace=True)

# Save the updated dataframe to a new CSV file
output_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\Call Type\updated_fiturid_uat.csv"
# ref_fiturid=ref_fiturid.iloc[62:63]
ref_fiturid.to_csv(output_path, index=False)


print(f"Updated CSV file saved to {output_path}")



Updated CSV file saved to C:\Users\maste\Downloads\dataloader_v60.0.2\Call Type\updated_fiturid_uat.csv


# Knowledge

## To see the characteristics of all files and save it as excel files

In [11]:
import pandas as pd
import os

# Function to find the longest length of values in each column
def find_longest_lengths_per_column(df):
    lengths = df.applymap(lambda x: len(str(x)) if pd.notnull(x) else 0)
    max_lengths = lengths.max()
    return max_lengths

# Set the folder path
folder_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\data-wi-produk-promo-program\samples_15_07_24"

# Get all Excel files in the folder
file_names = [file for file in os.listdir(folder_path) if file.endswith('.xlsx')]

# Load the files dynamically
dfs = []
rows_count = []
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_excel(file_path)
    df = df.drop(columns=['Unnamed: 0'], errors='ignore')
    dfs.append((file_name, df))
    rows_count.append((file_name, len(df)))

# Calculate the longest length for each column in each file
longest_lengths = {}
for file_name, df in dfs:
    longest_lengths[file_name] = find_longest_lengths_per_column(df)

# Combine results into a single DataFrame
results = pd.DataFrame(longest_lengths).transpose()

# Create DataFrame for rows count
rows_count_df = pd.DataFrame(rows_count, columns=["File Name", "Total Rows"])

# Print the results
results = results.transpose()
results.to_excel('allcolumns_artikel2.xlsx')

# Save rows count to a new Excel file
rows_count_df.to_excel('allcolumns_artikel_rows_count.xlsx', index=False)

# Find the longest column and its length for each file
longest_columns_info = []
for file_name, df in dfs:
    lengths = find_longest_lengths_per_column(df)
    longest_col = lengths.idxmax()
    longest_length = lengths.max()
    longest_columns_info.append({"File": file_name, "Column": longest_col, "Length": longest_length})

# Convert the list to a DataFrame
longest_columns_df = pd.DataFrame(longest_columns_info)
longest_columns_df.to_excel('allcolumns_artikel_col.xlsx', index=False)


C:\Users\maste\AppData\Local\Temp\ipykernel_32596\2866064037.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  lengths = df.applymap(lambda x: len(str(x)) if pd.notnull(x) else 0)
C:\Users\maste\AppData\Local\Temp\ipykernel_32596\2866064037.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  lengths = df.applymap(lambda x: len(str(x)) if pd.notnull(x) else 0)
C:\Users\maste\AppData\Local\Temp\ipykernel_32596\2866064037.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  lengths = df.applymap(lambda x: len(str(x)) if pd.notnull(x) else 0)
C:\Users\maste\AppData\Local\Temp\ipykernel_32596\2866064037.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  lengths = df.applymap(lambda x: len(str(x)) if pd.notnull(x) else 0)
C:\Users\maste\AppData\Local\Temp\ipykernel_32596\2866064037.py:6: FutureWarning: DataFrame.applymap has been deprecated

In [3]:
import pandas as pd
import os

# Function to find the longest length of values in each column
def find_longest_lengths_per_column(df):
    lengths = df.applymap(lambda x: len(str(x)) if pd.notnull(x) else 0)
    max_lengths = lengths.max()
    return max_lengths

# Set the folder path
# folder_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\sample_wise\sample_wise"

folder_path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\data-wi-produk-promo-program\samples_15_07_24"

# List of file names
file_names = [
    "sample_wi_100.xlsx",
    "sample_wise_produk_58.xlsx",
    "sample_wise_program_41.xlsx",
    "sample_wise_promo_90.xlsx"
]

# Load the files dynamically
dfs = []
rows_count = []
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_excel(file_path)
    df = df.drop(columns=['Unnamed: 0'], errors='ignore')
    dfs.append((file_name, df))
    rows_count.append((file_name, len(df)))

# Calculate the longest length for each column in each file
longest_lengths = {}
for file_name, df in dfs:
    longest_lengths[file_name] = find_longest_lengths_per_column(df)

# Combine results into a single DataFrame
results = pd.DataFrame(longest_lengths).transpose()

# Create DataFrame for rows count
rows_count_df = pd.DataFrame(rows_count, columns=["File Name", "Total Rows"])

# Print the results
results = results.transpose()
results.to_excel('allcolumns_artikel2.xlsx')

# Save rows count to a new Excel file
rows_count_df.to_excel('allcolumns_artikel_rows_count.xlsx', index=False)

# Optional: Find the longest column and its length for each file
# longest_columns_info = []
# for file_name, lengths in results.iterrows():
#     longest_col = lengths.idxmax()
#     longest_length = lengths.max()
#     longest_columns_info.append({"File": file_name, "Column": longest_col, "Length": longest_length})

# Convert the list to a DataFrame
# longest_columns_df = pd.DataFrame(longest_columns_info)
# longest_columns_df.to_excel('longest_columns_info.xlsx', index=False)


C:\Users\maste\AppData\Local\Temp\ipykernel_32596\2514165187.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  lengths = df.applymap(lambda x: len(str(x)) if pd.notnull(x) else 0)


## Working Instruction

In [50]:
import pandas as pd
from bs4 import BeautifulSoup

# Load the Excel file
path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\data-wi-produk-promo-program\samples_15_07_24\sample_wise_wi_900.xlsx"
df = pd.read_excel(path)

# Drop the 'Unnamed: 0' column if it exists
df = df.drop(columns=['Unnamed: 0'], errors='ignore')

# Function to clean text by removing HTML tags and unwanted characters
def clean_text(text):
    if isinstance(text, str):
        # Remove HTML tags
        clean_text = BeautifulSoup(text, "html.parser").get_text()
        # Replace \n and \t with actual new lines and tabs
        clean_text = clean_text.replace('\\n', '\n').replace('\\t', '\t')
        # Remove trailing and leading spaces
        clean_text = clean_text.strip()
        return clean_text
    return text

# Apply the clean_text function to all text columns
text_columns = df.select_dtypes(include=[object]).columns
df[text_columns] = df[text_columns].applymap(clean_text)

# Convert the date format
# df['modified_time'] = pd.to_datetime(df['modified_time']).dt.strftime('%Y-%m-%d %H:%M:%S')

# Function to generate sequential URL names
def generate_url_name(index):
    return f"WI-{index:07d}"

# Apply the function to generate URL names based on the row index + 1
df['URL_Name'] = [generate_url_name(i + 1) for i in range(len(df))]

# Check the 'calltype' column and replace "-" or empty values with "No Call Type"

df['Article_Type']='Working Instruction'
df['record_type']='Working Instructions'

# Add the new 'title' column
df.insert(0, 'title', df['calltype'] + ' - ' + df['kategori'] + ' - ' + df['sub_kategori'])
df['calltype'] = df['calltype'].apply(lambda x: 'No Call Type' if pd.isnull(x) or x.strip() == '-' else x)

# Save the cleaned DataFrame to a CSV file
df.to_csv('cleaned_wise_wi_900.csv', index=False)



C:\Users\maste\AppData\Local\Temp\ipykernel_36520\4243200304.py:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[text_columns] = df[text_columns].applymap(clean_text)
c:\Users\maste\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


## Product

In [49]:
import json
import pandas as pd
from bs4 import BeautifulSoup
from prettytable import PrettyTable
import openpyxl
from openpyxl.utils import get_column_letter
from openpyxl.drawing.image import Image
import matplotlib.pyplot as plt
import os

# Load the new Excel file
file_path_new = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\data-wi-produk-promo-program\samples_15_07_24\sample_wise_produk_75.xlsx"
df_program = pd.read_excel(file_path_new)

# Drop the 'Unnamed: 0' column if it exists
df_program = df_program.drop(columns=['Unnamed: 0'], errors='ignore')

# Function to clean text by removing HTML tags and unwanted characters
def clean_text(text):
    if isinstance(text, str):
        clean_text = BeautifulSoup(text, "html.parser").get_text()
        clean_text = clean_text.replace('\\n', '\n').replace('\\t', '\t')
        clean_text = clean_text.strip()
        return clean_text
    return text

# Add the new 'title' column
df_program.insert(0, 'title', df_program['jenis_produk'] + ' - ' + df_program['produk'] + ' - ' + df_program['varian_produk'])

# Function to generate sequential URL names
def generate_url_name(index):
    return f"PD-{index:07d}"

# Apply the function to generate URL names based on the row index + 1
df_program['URL_Name'] = [generate_url_name(i + 1) for i in range(len(df_program))]
df_program['calltype'] = 'No Call Type'
df_program['Article_Type'] = 'Product'
df_program['record_type'] = 'Product'

# Apply the clean_text function to all text columns
text_columns_program = df_program.select_dtypes(include=[object]).columns
df_program[text_columns_program] = df_program[text_columns_program].applymap(clean_text)

# Function to save DataFrame to Excel with auto-fit columns and borders
def save_df_to_excel(df, file_path):
    writer = pd.ExcelWriter(file_path, engine='openpyxl')
    df.to_excel(writer, index=False, sheet_name='Sheet1')
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']

    # Auto-fit columns
    for col in worksheet.columns:
        max_length = 0
        column = col[0].column_letter
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        worksheet.column_dimensions[column].width = adjusted_width

    # Add borders to cells
    thin_border = openpyxl.styles.Border(
        left=openpyxl.styles.Side(style='thin'),
        right=openpyxl.styles.Side(style='thin'),
        top=openpyxl.styles.Side(style='thin'),
        bottom=openpyxl.styles.Side(style='thin')
    )

    for row in worksheet.iter_rows():
        for cell in row:
            cell.border = thin_border

    workbook.save(file_path)

# Function to generate a picture of the DataFrame
def generate_df_picture(df, file_path):
    fig, ax = plt.subplots(figsize=(12, len(df) // 2))
    ax.axis('tight')
    ax.axis('off')
    table = ax.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)
    plt.savefig(file_path, bbox_inches='tight', pad_inches=0.1)
    plt.close()

# Function to check if a string is valid JSON and convert it to readable format
def process_json(text, row_idx, col_name, save_path, generate_pictures):
    try:
        data = json.loads(text)
        if isinstance(data, list) and 'table_data' in data[0]:
            # Save the JSON data to a separate Excel file
            table_data = data[0]['table_data']
            rows = []

            for row in table_data:
                row_values = [item['value'] for item in row]
                rows.append(row_values)

            json_df = pd.DataFrame(rows[1:], columns=rows[0])
            json_file_path = os.path.join(save_path, f'json_table_{row_idx}_{col_name}.xlsx')
            save_df_to_excel(json_df, json_file_path)

            if generate_pictures:
                # Generate picture of the table
                img_file_path = json_file_path.replace('.xlsx', '.png')
                generate_df_picture(json_df, img_file_path)

            table = PrettyTable()
            table.align = "l"  # Align all text to the left

            headers = [item['value'] for item in table_data[0]]
            table.field_names = headers
            
            # Calculate the maximum width for each column
            max_widths = {header: len(header) for header in headers}
            for row in table_data[1:]:
                for i, item in enumerate(row):
                    value_length = len(str(item['value']).strip())
                    if value_length > max_widths[headers[i]]:
                        max_widths[headers[i]] = value_length
            
            for row in table_data[1:]:
                row_values = [item['value'].strip() if isinstance(item['value'], str) else item['value'] for item in row]
                table.add_row(row_values)
            
            # Adjust column widths
            for field in table.field_names:
                table.max_width[field] = max_widths[field]
            
            return str(table)
        return text
    except (ValueError, TypeError, KeyError):
        return text

# Process and clean JSON-like text in all columns, saving JSON data to separate files
def process_all_json_texts(df, save_path, generate_pictures):
    json_columns_rows = []
    text_columns_program = df.select_dtypes(include=[object]).columns
    for column in text_columns_program:
        for idx, value in enumerate(df[column]):
            if isinstance(value, str) and value.strip().startswith('[') and value.strip().endswith(']'):
                df.at[idx, column] = process_json(value, idx, column, save_path, generate_pictures)
                json_columns_rows.append((column, idx))
    return json_columns_rows

# Define the path where to save JSON files and whether to generate pictures
save_path = r"D:\dataquality2\product_artikel"
generate_pictures = False

# Create the directory if it does not exist
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Process all JSON texts in the DataFrame
json_columns_rows = process_all_json_texts(df_program, save_path, generate_pictures)

# Save the cleaned DataFrame to a new CSV file
output_file_path = 'cleaned_product_75.csv'
df_program.to_csv(output_file_path, index=False)

# Display lines 39 to 42 for user reference
# lines_39_to_42 = df_program.iloc[38:42]
# print(lines_39_to_42['Biaya'].values)

# Add comments indicating columns and rows with JSON texts
for column, row in json_columns_rows:
    print(f"Column '{column}' at row {row} contains JSON text. Saved to '{os.path.join(save_path, f'json_table_{row}_{column}.xlsx')}'.")


Column 'Biaya' at row 39 contains JSON text. Saved to 'D:\dataquality2\product_artikel\json_table_39_Biaya.xlsx'.
Column 'Biaya' at row 59 contains JSON text. Saved to 'D:\dataquality2\product_artikel\json_table_59_Biaya.xlsx'.
Column 'Bunga' at row 59 contains JSON text. Saved to 'D:\dataquality2\product_artikel\json_table_59_Bunga.xlsx'.
Column 'Bunga' at row 67 contains JSON text. Saved to 'D:\dataquality2\product_artikel\json_table_67_Bunga.xlsx'.
Column 'Limit' at row 39 contains JSON text. Saved to 'D:\dataquality2\product_artikel\json_table_39_Limit.xlsx'.
Column 'Limit' at row 59 contains JSON text. Saved to 'D:\dataquality2\product_artikel\json_table_59_Limit.xlsx'.


C:\Users\maste\AppData\Local\Temp\ipykernel_36520\3705421417.py:42: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_program[text_columns_program] = df_program[text_columns_program].applymap(clean_text)
c:\Users\maste\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [25]:
# import pandas as pd
# from bs4 import BeautifulSoup

# # Load the new Excel file
# file_path_new = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\data-wi-produk-promo-program\samples_15_07_24\sample_wise_produk_75.xlsx"
# df_new = pd.read_excel(file_path_new)

# # Drop the 'Unnamed: 0' column if it exists
# df_new = df_new.drop(columns=['Unnamed: 0'], errors='ignore')

# # Function to clean text by removing HTML tags and unwanted characters
# def clean_text(text):
#     if isinstance(text, str):
#         # Remove HTML tags
#         clean_text = BeautifulSoup(text, "html.parser").get_text()
#         # Replace \n and \t with actual new lines and tabs
#         clean_text = clean_text.replace('\\n', '\n').replace('\\t', '\t')
#         # Remove trailing and leading spaces
#         clean_text = clean_text.strip()
#         return clean_text
#     return text

# # Apply the clean_text function to all text columns
# text_columns_new = df_new.select_dtypes(include=[object]).columns
# df_new[text_columns_new] = df_new[text_columns_new].applymap(clean_text)

# # Convert the date format
# # df_new['modified_time'] = pd.to_datetime(df_new['modified_time']).dt.strftime('%Y-%m-%d %H:%M:%S')

# # Add the new 'title' column
# df_new.insert(0, 'title', df_new['jenis_produk'] + ' - ' + df_new['produk'] + ' - ' + df_new['varian_produk'] + ' - ' + df_new['nama_knowledge'])

# # Function to generate sequential URL names
# def generate_url_name(index):
#     return f"PD-{index:07d}"

# # Apply the function to generate URL names based on the row index + 1
# df_new['URL_Name'] = [generate_url_name(i + 1) for i in range(len(df_new))]

# # Save the cleaned DataFrame to a CSV file
# df_new['calltype'] = 'No Call Type'
# df_new['Article_Type']='Product'
# df_new['record_type']='Product'
# # df_new=df_new.iloc[:1]
# df_new.to_csv('cleaned_wise_produk_75.csv', index=False)


## Program

In [54]:
import pandas as pd
from bs4 import BeautifulSoup

# Load the new Excel file
file_path_program = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\data-wi-produk-promo-program\samples_15_07_24\sample_wise_program_50.xlsx"
df_program = pd.read_excel(file_path_program)

# Drop the 'Unnamed: 0' column if it exists
df_program = df_program.drop(columns=['Unnamed: 0'], errors='ignore')

# Function to clean text by removing HTML tags and unwanted characters
def clean_text(text):
    if isinstance(text, str):
        # Remove HTML tags
        clean_text = BeautifulSoup(text, "html.parser").get_text()
        # Replace \n and \t with actual new lines and tabs
        clean_text = clean_text.replace('\\n', '\n').replace('\\t', '\t')
        # Remove trailing and leading spaces
        clean_text = clean_text.strip()
        return clean_text
    return text

# Apply the clean_text function to all text columns
text_columns_program = df_program.select_dtypes(include=[object]).columns
df_program[text_columns_program] = df_program[text_columns_program].applymap(clean_text)

# Convert the date format for 'modified_time', 'start', and 'end' columns
# df_program['modified_time'] = pd.to_datetime(df_program['modified_time']).dt.strftime('%Y-%m-%d %H:%M:%S')
df_program['start'] = pd.to_datetime(df_program['start'], errors='coerce').dt.strftime('%Y-%m-%d')
df_program['end'] = pd.to_datetime(df_program['end'], errors='coerce').dt.strftime('%Y-%m-%d')

# Add the new 'title' column
df_program.insert(0, 'title', df_program['nama_program'])

# Function to generate sequential URL names
def generate_url_name(index):
    return f"PG-{index:07d}"

# Apply the function to generate URL names based on the row index + 1
df_program['URL_Name'] = [generate_url_name(i + 1) for i in range(len(df_program))]

# Save the cleaned DataFrame to a CSV file
df_program['calltype'] = 'No Call Type'
df_program['Article_Type']='Program'
df_program['record_type']='Program'
# df_program=df_program.iloc[:1]
df_program.to_csv('cleaned_wise_program_50.csv', index=False)

C:\Users\maste\AppData\Local\Temp\ipykernel_36520\2102008483.py:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_program[text_columns_program] = df_program[text_columns_program].applymap(clean_text)
c:\Users\maste\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


## Promo

In [57]:
import pandas as pd
from bs4 import BeautifulSoup

# Load the new Excel file
file_path_promo = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\data-wi-produk-promo-program\samples_15_07_24\sample_wise_promo_107.xlsx"
df_promo = pd.read_excel(file_path_promo)

# Drop the 'Unnamed: 0' column if it exists
df_promo = df_promo.drop(columns=['Unnamed: 0'], errors='ignore')

# Function to clean text by removing HTML tags and unwanted characters
def clean_text(text):
    if isinstance(text, str):
        # Remove HTML tags
        clean_text = BeautifulSoup(text, "html.parser").get_text()
        # Replace \n and \t with actual new lines and tabs
        clean_text = clean_text.replace('\\n', '\n').replace('\\t', '\t')
        # Remove trailing and leading spaces
        clean_text = clean_text.strip()
        return clean_text
    return text

# Apply the clean_text function to all text columns
text_columns_promo = df_promo.select_dtypes(include=[object]).columns
df_promo[text_columns_promo] = df_promo[text_columns_promo].applymap(clean_text)

# Convert the date format for 'modified_time', 'start', and 'end' columns
# df_promo['modified_time'] = pd.to_datetime(df_promo['modified_time']).dt.strftime('%Y-%m-%d %H:%M:%S')
df_promo['start'] = pd.to_datetime(df_promo['start'], errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')
df_promo['end'] = pd.to_datetime(df_promo['end'], errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')

# Add the new 'title' column
df_promo.insert(0, 'title', df_promo['kategori'])

# Function to generate sequential URL names
def generate_url_name(index):
    return f"PM-{index:07d}"

# Apply the function to generate URL names based on the row index + 1
df_promo['URL_Name'] = [generate_url_name(i + 1) for i in range(len(df_promo))]

# Save the cleaned DataFrame to a CSV file
df_promo['calltype'] = 'No Call Type'
df_promo['Article_Type']='Promotion'
df_promo['record_type']='Promotion'

# Save the cleaned DataFrame to a CSV file
# df_promo=df_promo.iloc[:1]
df_promo.to_csv('cleaned_wise_promo_90.csv', index=False)


C:\Users\maste\AppData\Local\Temp\ipykernel_36520\698870232.py:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_promo[text_columns_promo] = df_promo[text_columns_promo].applymap(clean_text)
c:\Users\maste\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [1]:
import pandas as pd

path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\knowledge\BRI - Detail BRICare(WI).csv"
df=pd.read_csv(path, encoding='ISO-8859-1')
df = df.fillna('')
df['Article Type']='Working Instruction'

output=r"C:\Users\maste\Downloads\dataloader_v60.0.2\knowledge\artikel_all.csv"



# Change 'Brilink' to 'BRILink' in 'Type' column
df['Type'] = df['Type'].replace('Brilink', 'BRILink')

# Remove rows with 'Wholesale' in 'Type' column
df = df[df['Type'] != 'Wholesale']

# df['Type'].unique()

df['URL_name'] = df['Call Type'].astype(str) + '-' + df['Call Type Info'].astype(str)

import re

# Define a function to clean the URL name
def clean_url_name(url_name):
    # Remove leading and trailing hyphens
    url_name = url_name.strip('-')
    # Replace invalid characters with hyphens and remove multiple hyphens
    url_name = re.sub(r'[^a-zA-Z0-9\u00C0-\u017F-]', '-', url_name)
    url_name = re.sub(r'-+', '-', url_name)
    # Remove leading and trailing hyphens again after replacement
    url_name = url_name.strip('-')
    return url_name

# Apply the cleaning function to the URL_name column
df['URL_name'] = df['URL_name'].apply(clean_url_name)



df.to_csv('artikel2.csv', index= False)
# df.iloc[:1].to_csv(output, index= False)

df



,Unnamed: 0,Call Type,Call Type Info,Type,User,Title,Detail Bricare,Deskripsi,Empati,Konfirmasi,...,Verifikasi,Gali Informasi,Pembuatan Laporan,Konfirmasi Ulang,Percepatan Komplain,"Solusi, Informasi",Edukasi & Cross Selling,Closing,Article Type,URL_name
0,1,1000,Nasabah Menanyakan Informasi Pengajuan Terkait...,CERIA,Agent Contact Center,1000 - Nasabah Menanyakan Informasi Pengajuan ...,nomor ponsel nasabah yang bisa dihubungi\nID C...,Nasabah menghubungi Contact BRI / datang ke Un...,Empati sesuai kondisi (pilih salah satu)\t\t\t...,Jika Nasabah menanyakan Cara & Syarat Pengajua...,...,1\tCara & Syarat pengajuan Aplikasi\nVerifikas...,1\tCara & Syarat pengajuan Aplikasi\nGali Info...,1\tCara & Syarat pengajuan Aplikasi\nBuat lapo...,,,1\tCara & Syarat pengajuan Aplikasi\nInformasi...,,Closing\t\t\n1\tFeedback\t\n\tApakah informasi...,Working Instruction,1000-Nasabah-Menanyakan-Informasi-Pengajuan-Te...
1,2,1002,Nasabah Menyakan Terkait Promo dan Program CERIA,CERIA,Agent Contact Center,1002 - Nasabah Menyakan Terkait Promo dan Prog...,,Nasabah menghubungi Contact BRI / datang ke Un...,Empati sesuai kondisi (pilih salah satu)\nJika...,Konfirmasi :\nJika Nasabah menanyakan promo ce...,...,Verifikasi\nTidak dilakukan Verifikasi\n,Gali Informasi\t\n\t i. Jenis promo/...,Pembuatan Laporan\t\n\ti. Nomor laporan ti...,,,Informasi & Solusi \t\t\n\t i. \...,,Closing\t\t\n1\tFeedback\t\n\tApakah informasi...,Working Instruction,1002-Nasabah-Menyakan-Terkait-Promo-dan-Progra...
2,3,1003,Nasabah Mengajukan Pelunasan Awal Cicilan CERIA,CERIA,Agent Contact Center,1003 - Nasabah Mengajukan Pelunasan Awal Cicil...,,Nasabah menghubungi Contact BRI / datang ke Un...,"Empati sesuai kondisi (pilih salah satu)\n""Bap...","Konfirmasi :\n""Untuk Permintaan Pelunasan awal...",...,Verifikasi ( Buka CLOS/ Finnachel )\t\nUntuk v...,Gali Informasi\t\n1.\tTanyakan alasan pelunasa...,Pembuatan Laporan \t\n1.\tBuat Laporan dengan ...,,,Informasi & Solusi\n1.\tCicilan telah berjalan...,,Closing\t\t\n1\tFeedback\t\n\tApakah informasi...,Working Instruction,1003-Nasabah-Mengajukan-Pelunasan-Awal-Cicilan...
3,4,1005,Nasabah Mengajukan Pemblokiran CERIA,CERIA,Agent Contact Center,1005 - Nasabah Mengajukan Pemblokiran CERIA,Nasabah mengajukan pemblokiran Sementara Akun ...,Nasabah menghubungi Contact BRI / datang ke Un...,Empati\nJika nasabah infokan ingin melakukan p...,"Konfirmasi: \n""Untuk pemblokiran bersifat seme...",...,Verifikasi ( Buka WBS/ CLOS )\t\nUntuk verifik...,Gali Informasi\t\n1.\tTanyakan alasan perminta...,Pembuatan Laporan\nNasabah mengajukan pembloki...,,,Informasi & Solusi \t\n1.\tAgent Konfirmasi ke...,"Kalimat Edukasi\n""Kami informasikan Bank BRI t...",Closing\t\t\n1\tFeedback\t\n\tApakah informasi...,Working Instruction,1005-Nasabah-Mengajukan-Pemblokiran-CERIA
4,5,1008,Nasabah Mengajukan Pengaktifan Akun Ceria Terb...,CERIA,Agent Contact Center,1008 - Nasabah Mengajukan Pengaktifan Akun Cer...,,,,,...,,,,,,,,,Working Instruction,1008-Nasabah-Mengajukan-Pengaktifan-Akun-Ceria...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,,,,,,,,,,,...,,,,,,,,,Working Instruction,
438,,,,,,,,,,,...,,,,,,,,,Working Instruction,
439,,,,,,,,,,,...,,,,,,,,,Working Instruction,
440,,,,,,,,,,,...,,,,,,,,,Working Instruction,


In [91]:
import pandas as pd

pd.set_option('display.max_columns', None)
df= pd.read_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\log\error060424041601915.csv")
df.to_csv('error.csv', index=False)

In [23]:
import pandas as pd


path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\knowledge\BRI - Detail BRICare 2.xlsx"

df = pd.read_excel(path)
df=df.iloc[:1]
# df['Gali Informasi']

df['URL_name'] = df['Call Type'].astype(str) + '-' + df['Call Type Info'].astype(str)

import re

# Define a function to clean the URL name
def clean_url_name(url_name):
    # Remove leading and trailing hyphens
    url_name = url_name.strip('-')
    # Replace invalid characters with hyphens and remove multiple hyphens
    url_name = re.sub(r'[^a-zA-Z0-9\u00C0-\u017F-]', '-', url_name)
    url_name = re.sub(r'-+', '-', url_name)
    # Remove leading and trailing hyphens again after replacement
    url_name = url_name.strip('-')
    return url_name

# Apply the cleaning function to the URL_name column
df['URL_name'] = df['URL_name'].apply(clean_url_name)

df.to_csv('artikel_bullet.csv', index=False)

# Generate Roles


## generate 1k roles

In [1]:
import pandas as pd

# Load the uploaded CSV file
file_path = '/mnt/data/extract_role.csv'
roles_df = pd.read_csv(file_path)

# Display the content of the CSV file to understand its structure
roles_df.head()


# Number of additional roles needed to reach a total of 1000
num_additional_roles_needed = 1000 - len(expanded_roles_df)

# Duplicate the existing roles to meet the required number of roles
additional_roles_more = expanded_roles_df.sample(num_additional_roles_needed, replace=True).reset_index(drop=True)

# Generate unique names for the additional roles
additional_roles_more['NAME'] = additional_roles_more['NAME'] + "_more_" + (additional_roles_more.index + 1).astype(str)

# Combine the original roles with the additional roles
expanded_roles_df_more = pd.concat([expanded_roles_df, additional_roles_more], ignore_index=True)

# Ensure we have a total of 1000 roles
expanded_roles_df_more = expanded_roles_df_more.head(1000)

# Save the expanded roles to a new CSV file
output_file_path_more = 'expanded_roles_1000.csv'
expanded_roles_df_more.to_csv(output_file_path_more, index=False)

output_file_path_more



FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/extract_role.csv'

## Check Extraction Data



### check data before 2023 = 27 columns

In [13]:
column_list = [
    "Ticket_ID",  
    "Call_Type_ID",  
    "Call_Type", 
    "Create_Date",  
    "gateway",  
    "Jenis_Laporan",  
    "Nama_Nasabah",  
    "No_Rekening", 
    "Nominal",  
    "status",  
    "TanggalClosed", 
    "tanggalTransaksi",  
    "Chanel",  
    "Fitur",  
    "Nomor_Kartu", 
    "user_group",  
    "assgined_to",  
    "attachment_done",  
    "email",  
    "full_name",  
    "no_telepon",  
    "approver_login",  
    "approver_name",  
    "SLAResolution",  
    "submitter_login_id",  
    "submitter_user_group", 
    "user_login_name"  
]

column_names = [
    "Ticket_ID", "Call_Type_ID", "Call_Type", "Create_Date", "gateway", "Jenis_Laporan", "Nama_Nasabah", 
    "No_Rekening", "Nominal", "status", "TanggalClosed", "tanggalTransaksi", "Chanel", "Fitur", "Nomor_Kartu", 
    "user_group", "assgined_to", "attachment_done", "email", "full_name", "no_telepon", "approver_login", 
    "approver_name", "SLAResolution", "submitter_login_id", "submitter_user_group", "user_login_name", 
    "Jenis_Produk", "Last_Modified_By", "Merchant_ID", "Modified_Date", "NOTAS", "Produk", "SLA_Status", "TID", 
    "tanggalAttachmentDone", "Tgl_Assigned", "Tgl_Eskalasi", "AnalisaSkils", "Attachment_", "Bank_BRI", 
    "Biaya_Admin", "Suku_Bunga", "Bunga", "Butuh_Attachment", "Cicilan", "Hasil_Kunjungan", "Log_Name", 
    "MMS_Ticket_Id", "Mass_Ticket_Upload_Flag", "Nama_Supervisor", "Nama_TL", "Nama_Wakabag", "Nasabah_Prioritas", 
    "Notify_By", "Organization", "Output_Settlement", "phone_survey", "Return_Ticket", "Settlement_By", 
    "Settlement_ID", "Settlement", "Site_User", "Status_Return", "Status_Transaksi", "Submitter_Region", 
    "Submitter_SiteGroup", "Submitter_User_group_ID", "Tanggal_Settlement", "Tgl_Foward", "Tgl_In_Progress", 
    "Tgl_Returned", "Ticket_Referensi", "Tiket_Urgency", "Tipe_Remark", "UniqueID", "users", "Usergroup_ID"
]

import pandas as pd

path=r"D:\dataquality2\new as per 5 June\bricare_20221213_20240604_27_kolom.csv"
path2=r"D:\dataquality2\new as per 5 June\bricare_20221213_20240604_79_kolom.csv"
# df= pd.read_csv(path, delimiter=';')
df= pd.read_csv(path2, delimiter=';')


# column_list_set = set(column_list)
column_list_set = set(column_names)
df_columns_set = set(df.columns)

missing_columns = column_list_set - df_columns_set
extra_columns = df_columns_set - column_list_set

if not missing_columns and not extra_columns:
    print("The column names match.")
else:
    if missing_columns:
        print(f"Missing columns in DataFrame: {missing_columns}")
    if extra_columns:
        print(f"Extra columns in DataFrame: {extra_columns}")

################################################
# column_list_set2 = set(column_names)
# df_columns_set2 = set(df2.columns)

# missing_columns2 = column_list_set2 - df_columns_set2
# extra_columns2 = df_columns_set2 - column_list_set2

# if not missing_columns and not extra_columns:
#     print("The column names match.")
# else:
#     if missing_columns2:
#         print(f"Missing columns in DataFrame: {missing_columns2}")
#     if extra_columns2:
#         print(f"Extra columns in DataFrame: {extra_columns2}")


Missing columns in DataFrame: {'Tgl_Assigned'}
Extra columns in DataFrame: {'TglAssigned', 'cifno', 'Details'}


Data dengan 27 kolom:
- jumlah kolom sudah ok
- picklist Walk In harus diganti ke Walk-In
- attachment
- penambahan nama file diakhir "27"



Data dengan 79 kolom:
- picklist Walk In harus diganti ke Walk-In
- penambahan nama file diakhir "79"



Data Zendesk:
- harus hilangkan double quotes ""
- delimiternya ;
- nama file zendesk


Data Omni:
- SMS: harus format csv

In [2]:
import pandas as pd

path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\log\error060524083553095.csv"
df=pd.read_csv(path)
df['ERROR']

0    Unable to create/update fields: LastModifiedDa...
1    Unable to create/update fields: LastModifiedDa...
2    Unable to create/update fields: LastModifiedDa...
3    Unable to create/update fields: LastModifiedDa...
4    Unable to create/update fields: LastModifiedDa...
5    Unable to create/update fields: LastModifiedDa...
6    Unable to create/update fields: LastModifiedDa...
7    Unable to create/update fields: LastModifiedDa...
8    Unable to create/update fields: LastModifiedDa...
9    Unable to create/update fields: LastModifiedDa...
Name: ERROR, dtype: object

# Zendesk

In [33]:
import pandas as pd

file_path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\data check\new as per 5 June\Data Zendesk 1-15 Januari 2024 - sampel.csv"

import csv


# Read the CSV file with proper handling of quotes and custom separator
with open(file_path, 'r', newline='') as file:
    reader = csv.reader(file, delimiter=',', quotechar='"')
    rows = [row for row in reader]

# Split the combined columns that have commas within them
split_rows = []
for row in rows:
    split_row = row[0].split(',') + row[1:]  # Split the first column and add the rest as they are
    split_rows.append(split_row)

# Create DataFrame from the processed rows
df = pd.DataFrame(split_rows[1:], columns=split_rows[0])

# Remove double quotes from column names
df.columns = df.columns.str.replace('"', '')

# Remove double quotes from all data in the DataFrame
df = df.replace('"', '', regex=True)

# Display the first few rows of the cleaned DataFrame
print(df.head())
df

# If you need to save the cleaned DataFrame to a new CSV file
# cleaned_file_path = '/mnt/data/Cleaned_Zendesk_Data.csv'
# df.to_csv(cleaned_file_path, index=False)

# print(f"Cleaned data saved to {cleaned_file_path}")



  Ticket ID Ticket channel   Assignee ID   Assignee name    Requester ID  \
0   3435272    Any channel  405257525413  Agent Sosmed 4  27064719563033   
1   3435273       Facebook  405257335633  Agent Sosmed 3  27064768687769   

     Requester name                                     Ticket subject  \
0        vianovia94  [IGC] Minimal opo boloo... ......#relsvideo #r...   
1  Masruroh Sodikin                                                  P   

  Requester created - Timestamp Ticket created - Timestamp  \
0           2024-01-01T00:03:30        2024-01-01T00:03:30   
1           2024-01-01T00:04:15        2024-01-01T00:04:15   

  Ticket solved - Timestamp                 Tickets  
0       2024-01-01T00:30:56  1.00000000000000000000  
1       2024-01-01T00:46:27  1.00000000000000000000  


,Ticket ID,Ticket channel,Assignee ID,Assignee name,Requester ID,Requester name,Ticket subject,Requester created - Timestamp,Ticket created - Timestamp,Ticket solved - Timestamp,Tickets
0,3435272,Any channel,405257525413,Agent Sosmed 4,27064719563033,vianovia94,[IGC] Minimal opo boloo... ......#relsvideo #r...,2024-01-01T00:03:30,2024-01-01T00:03:30,2024-01-01T00:30:56,1.00000000000000000000
1,3435273,Facebook,405257335633,Agent Sosmed 3,27064768687769,Masruroh Sodikin,P,2024-01-01T00:04:15,2024-01-01T00:04:15,2024-01-01T00:46:27,1.00000000000000000000


In [2]:
import pandas as pd


path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\data check\new as per 5 June\bricare_20221213_20240604_case_account.csv"
# df=pd.read_csv(path)
# df



path2=r"C:\Users\maste\Downloads\dataloader_v60.0.2\data check\new as per 5 June\case.csv"
df=pd.read_csv(path2)
df

ParserError: Error tokenizing data. C error: Expected 2 fields in line 86, saw 3


### Dummy data for Alex7

# Data Account Alex7

In [7]:
# Create dummy data for account with quoting (DONE)

import csv
import random
import os

def generate_dummy_data(file_path, file_name, num_rows):
    # Define possible values for each column
    account_names = ['John Doe', 'Jane Smith', 'Mike Brown', 'Lisa Green', 'Mark Taylor']
    account_owners = ['Owner A', 'Owner B', 'Owner C', 'Owner D']
    nasabah_types = ['Nasabah', 'Non Nasabah']
    account_record_types = ['Personal', 'Non Personal']

    # Ensure the directory exists
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    
    # Writing to the CSV file
    with open(os.path.join(file_path, file_name), mode='w', newline='') as file:
        writer = csv.writer(file, quotechar='"', quoting=csv.QUOTE_ALL)  # Enforce quoting for all fields
        writer.writerow(['Account Name', 'CIF No', 'Account Owner', 'No Telp', 'Email', 'Nasabah Type', 'Account Record Type'])
        
        for _ in range(num_rows):
            account_name = random.choice(account_names)
            email = f"{account_name.split(' ')[0].lower()}.{account_name.split(' ')[1].lower()}@example.com"
            writer.writerow([
                account_name,
                ''.join(["{}".format(random.randint(0, 9)) for _ in range(10)]),  # 10-digit CIF No
                random.choice(account_owners),
                f'+62{random.randint(1000000000, 9999999999)}',  # Phone number
                email,
                random.choice(nasabah_types),
                random.choice(account_record_types)
            ])


file_path = 'D:\dataquality2'  # Adjust the path as needed
file_name = 'dummy_data_account.csv'
num_rows = 1  # Adjust the number of rows as needed

generate_dummy_data(file_path, file_name, num_rows)


## Data Case ALex 7


num of rows 

In [33]:
def create_salesforce_case_dummy_data(num_rows=100, start_ttb=1):
    import pandas as pd
    import random

    # Define the columns and possible values
    statuses = ["New", "Working", "Escalated"]
    types = ["Electronic", "Electrical", "Mechanical"]
    case_reasons = ["Performance", "Breakdown"]

    # Generate sample data
    data = {
        "Status": [random.choice(statuses) for _ in range(num_rows)],
        "Type": [random.choice(types) for _ in range(num_rows)],
        "Case Reason": [random.choice(case_reasons) for _ in range(num_rows)],
        "Subject": [f"Subject {i+1}" for i in range(num_rows)],
        "Description": [f"Description of issue {i+1}" for i in range(num_rows)],
        "Legacy_Ticket_ID": [f"TTB{start_ttb + i:06d}" for i in range(num_rows)],  # Generating 6-digit random numbers in order
    }

    # Create DataFrame
    df = pd.DataFrame(data)

    # Save to CSV
    file_path = 'dummy_casefor_alex11.csv'
    df['record_type'] = 'Case Migration'
    df.to_csv(file_path, index=False)

    return file_path

# Example usage
file_path = create_salesforce_case_dummy_data(num_rows=5, start_ttb=1006)
file_path


'dummy_casefor_alex11.csv'

In [4]:
import pandas as pd

path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\bricaredatareal_20200101_20200201.csv"

df=pd.read_csv(path)
df=df.iloc[:100]
df.to_csv('bricare_100_pakpondah.csv',index=False)

# data dummy



C:\Users\maste\AppData\Local\Temp\ipykernel_5108\2256201397.py:5: DtypeWarning: Columns (14,16,21,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(path)


In [1]:
import pandas as pd
import math

def generate_dummy_files(base_filename, total_rows, start_ttb='TTB0000000001'):
    
    rows_per_file = 500000
    num_files = math.ceil(total_rows / rows_per_file)
    
    
    start_number = int(start_ttb[3:])
    
    for file_index in range(num_files):
       
        if file_index == num_files - 1:
            current_rows = total_rows % rows_per_file or rows_per_file
        else:
            current_rows = rows_per_file
        
        
        data = {
            'Status': ['Cancelled'] * current_rows,
            'Priority': ['Low'] * current_rows,
            'Call Type': ['1000'] * current_rows,
            'record_type': ['Case Migration'] * current_rows,
            'Legacy_ticket_id': [f'TTB{str(i).zfill(10)}' for i in range(start_number + file_index * rows_per_file, start_number + file_index * rows_per_file + current_rows)]
        }
        
       
        df = pd.DataFrame(data)
        
       
        current_filename = f"{base_filename}_part{file_index + 1}.csv"
        
        
        df.to_csv(current_filename, index=False)
        print(f"{current_filename} with {current_rows} rows created successfully.")
    


base_filename = 'dummy_data_casenumber60'
total_rows = 500000
start_ttb = 'TTB0000000000'
generate_dummy_files(base_filename, total_rows, start_ttb)


dummy_data_casenumber60_part1.csv with 500000 rows created successfully.


In [45]:
import pandas as pd

path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\Case2.csv"

df =pd.read_csv(path)
df = df[21:]
df['New_SCC_LEGACY_TICKET_ID__C']=df['CASENUMBER']
df
df.to_csv('case_okta.csv', index=False)
df

,CASENUMBER,SCC_LEGACY_TICKET_ID__C,New_SCC_LEGACY_TICKET_ID__C
21,TTB0000092801,TTB0005092801,TTB0000092801
22,TTB0000092802,TTB0005092802,TTB0000092802
23,TTB0000092803,TTB0005092803,TTB0000092803
24,TTB0000092804,TTB0005092804,TTB0000092804
25,TTB0000092805,TTB0005092805,TTB0000092805
...,...,...,...
1714216,TTB0001714196,TTB0006714196,TTB0001714196
1714217,TTB0001714197,TTB0006714197,TTB0001714197
1714218,TTB0001714198,TTB0006714198,TTB0001714198
1714219,TTB0001714199,TTB0006714199,TTB0001714199


## Data Case ALex 7
Join the two files based on Ticket ID

In [18]:
#extract the ID and CaseNumber:
# SELECT Id, CaseNumber
# FROM Case 
# WHERE CreatedById = '005MR0000004GJ7YAM'

# from the real data to join the ID or Case Number and update the records in Case 

import pandas as pd

# Load the data from the uploaded files
extract_df = pd.read_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\extract_150_casenumberandID.csv")
# test_data_dummy_df = pd.read_csv(r"C:\Users\maste\Downloads\Test Data Dummy.csv")
test_data_dummy_df = pd.read_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\log\error062124052713145.csv")


# Split the 'ID;"CASENUMBER"' column into two separate columns
extract_df[['ID', 'CASENUMBER']] = extract_df['ID;"CASENUMBER"'].str.split(';', expand=True)

# Drop the original combined column
extract_df.drop(columns=['ID;"CASENUMBER"'], inplace=True)

# Remove leading and trailing quotes from the columns
extract_df['ID'] = extract_df['ID'].str.strip('"')
extract_df['CASENUMBER'] = extract_df['CASENUMBER'].str.strip('"')

# Rename columns for easier merging
extract_df.rename(columns={'CASENUMBER': 'Ticket_ID'}, inplace=True)

# Merge the dataframes based on 'Ticket_ID'
merged_df = test_data_dummy_df.merge(extract_df, on='Ticket_ID', how='left')

# Reorder the columns to place 'ID' as the first column
columns = ['ID'] + [col for col in merged_df.columns if col != 'ID']
merged_df = merged_df[columns]

# Save the updated dataframe to a new CSV file
# merged_df=merged_df.iloc[:1]
merged_df['Call_Type_ID']='1000'
merged_df=merged_df.drop('ID', axis=1)
merged_df.to_csv('Updated_Test_Data_Dummy_2row.csv', index=False)


KeyError: 'Ticket_ID'

In [38]:
# the extracted data from Salesforce after being updated 


import pandas as pd

pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\case_150.csv"
path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\extract_case_uat_120k.csv"

df=pd.read_csv(path, delimiter=';')
df2 = df[['ID', 'STATUS','CASENUMBER', 'SCC_LEGACY_TICKET_ID__C']]

# df3 = df2[df2['SCC_LEGACY_TICKET_ID__C'].notna()]
# df2.iloc[70990:]





C:\Users\maste\AppData\Local\Temp\ipykernel_17788\1435779221.py:11: DtypeWarning: Columns (47,48,84,108,124,130,132,189,200,203,204,229,247) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(path, delimiter=';')


## Update the Supported Objects in Queue Name



In [35]:
import pandas as pd

# First Extract all the Queue ID from Group Object, you may use this query:
# SELECT Id
# where Type ='Queue'


path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\extract_group_queueid.csv"

df = pd.read_csv(path)

# add one column 'SObjectType' = Case, see the example below:
# QueueId,SObjectType
# 00GMR0000000trT2AQ,Case

df['SObjectType']='Case'
df.to_csv('extract_group_queueid.csv', index=False)

# Go to Data Loader 'insert' and choose  'QueueSobject' and insert the file with two columns: Id and SObjectType = Case

In [1]:
import pandas as pd


path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\log\error062124041536955.csv"


df = pd.read_csv(path)
# df= df.drop('ERROR', axis=1)
# df= df.drop('ERROR.1', axis=1)
df.to_csv(path, index=False)

In [41]:
import pandas as pd

path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\sample_case (2)\bricare_20200806_20240430_0_27.csv"
df = pd.read_csv(path, delimiter=';')
df.columns

Index(['cifno', 'Ticket_ID', 'Call_Type_ID', 'Call_Type', 'Create_Date',
       'gateway', 'Jenis_Laporan', 'Nama_Nasabah', 'No_Rekening', 'Nominal',
       'status', 'TanggalClosed', 'tanggalTransaksi', 'Chanel', 'Fitur',
       'Nomor_Kartu', 'user_group', 'assgined_to', 'attachment_done', 'email',
       'full_name', 'no_telepon', 'approver_login', 'approver_name',
       'SLAResolution', 'submitter_login_id', 'submitter_user_group',
       'user_login_name'],
      dtype='object')

In [45]:
import pandas as pd
import numpy as np

# Define the file path
# path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\sample_case (2)\bricare_20200806_20240430_0_27.csv"
path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\sample_case (2)\bricare_20200806_20240430_1000001_27.csv"

# Load the data
df = pd.read_csv(path, delimiter=';')

# df = df.iloc[:1]
# Change the year to 2019
df['Create_Date'] = pd.to_datetime(df['Create_Date'])
df['Create_Date'] = df['Create_Date'].apply(lambda x: x.replace(year=2019))

def generate_random_16_digits():
    return ''.join([str(np.random.randint(0, 10)) for _ in range(16)])

# Add a new column 'Nomor_Kartu' with random 16-digit numbers
df['Nomor_Kartu'] = df.apply(lambda x: generate_random_16_digits(), axis=1)

# Remove the .0 from Call_Type_ID by converting it to an integer
df['Call_Type_ID'] = '8412'
df['Call_Type_ID'] = df['Call_Type_ID'].astype(int)

def generate_random_15_to_18_digits():
    return ''.join([str(np.random.randint(0, 10)) for _ in range(np.random.randint(15, 19))])

# Update the 'No_Rekening' column with random 15 to 18 digit numbers
df['No_Rekening'] = df.apply(lambda x: generate_random_15_to_18_digits(), axis=1)

# add recordtype
df['record_type']='Case Migration'

df.to_csv('bricare_27col_100k_2.csv', index=False)


# df=df[df['Ticket_ID'] == 'TTB000025269322']
# df

# Extract just the 'Ticket_ID' column and sort it
# df2 = df['Ticket_ID'].sort_values(ascending=True)
# df2

# Display the sorted Series
# df2.to_csv('tiket_bricare.csv', index=False)


#dummy 120k 

In [21]:
import pandas as pd

path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\bricare_27col_100k.csv"

df=pd.read_csv(path)



df['Ticket_ID'] = df['Ticket_ID'].str.replace('TTB', 'TICKET')

df.to_csv('bricare_120k_ticket.csv', index=False)

In [22]:
import pandas as pd

# Load the data files
extract_df = pd.read_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\extract_dummyfromserver_120k.csv.xls")
bricare_df = pd.read_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\bricare_27col_100k.csv")

# Check the lengths of the CaseNumber and Ticket_ID columns
extract_length = len(extract_df['CaseNumber'])
bricare_length = len(bricare_df['Ticket_ID'])

# Ensure the number of CaseNumber matches the number of Ticket_ID
if extract_length >= bricare_length:
    # Trim the extract dataframe to match the length of the bricare dataframe
    trimmed_extract_df = extract_df.iloc[:bricare_length]
    
    # Replace Ticket_ID values with CaseNumber values
    bricare_df['Ticket_ID'] = trimmed_extract_df['CaseNumber'].values
    
    # Save the updated bricare dataframe to a new CSV file
    output_file_path = 'updated_bricare_120k_ticket.csv'
    # bricare_df.to_csv(output_file_path, index=False)
    
    print("Ticket_ID values have been successfully replaced and saved to 'updated_bricare_120k_ticket.csv'.")
else:
    print("Error: The number of CaseNumber entries is less than the number of Ticket_ID entries. Ensure they are one-to-one.")

# Display the first few rows of the updated bricare dataframe
bricare_df.head()


Ticket_ID values have been successfully replaced and saved to 'updated_bricare_120k_ticket.csv'.


,cifno,Ticket_ID,Call_Type_ID,Call_Type,Create_Date,gateway,Jenis_Laporan,Nama_Nasabah,No_Rekening,Nominal,...,email,full_name,no_telepon,approver_login,approver_name,SLAResolution,submitter_login_id,submitter_user_group,user_login_name,record_type
0,LD86852,TICKET000998035,8412,Nasabah BRI Gagal Transaksi Belanja di EDC BRI...,2019-08-06 16:52:14,Phone,Complaint - Transaction,b2714defcfef8c65d3154367beb604aa,878667067646102,23300800.0,...,NaN,d41d8cd98f00b204e9800998ecf8427e,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,20.0,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,Case Migration
1,RCAW297,TICKET000998036,8412,Nasabah BRI gagal tarik tunai & terdebet di AT...,2019-08-06 16:52:14,Phone,Complaint - Transaction,b69e3868bc8b6c3231df7d691a5595fe,5073214431798791,1000000.0,...,NaN,d41d8cd98f00b204e9800998ecf8427e,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,10.0,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,Case Migration
2,AOY4038,TICKET000998037,8412,Nasabah BRI Gagal Transaksi Belanja di EDC BRI...,2019-08-06 16:52:14,Phone,Complaint - Transaction,948421d448fb51f1cf27584b1256a120,6430217575043687,246300.0,...,NaN,d41d8cd98f00b204e9800998ecf8427e,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,20.0,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,Case Migration
3,AUA0263,TICKET000998038,8412,Nasabah BRI gagal transfer & terdebet jaringan...,2019-08-06 16:52:14,Phone,Complaint - Transaction,a4aa532c65d1503dcc41ef396d3c42b8,4067829670996616,100000.0,...,NaN,d41d8cd98f00b204e9800998ecf8427e,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,20.0,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,Case Migration
4,HOL2556,TICKET000998039,8412,Kartu ATM BRI Tertelan di MESIN ATM,2019-08-06 16:52:14,Phone,Request,95d4f5421b5b387726b5d619dbe24155,272778316775473552,0.0,...,NaN,fcc2dc136c7c3d69fb080cdd2b1648f1,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,20.0,90119221.0,LCC-CCTCALL,fcc2dc136c7c3d69fb080cdd2b1648f1,Case Migration


In [30]:
import pandas as pd

path=r"D:\dataquality2\updated_bricare_120k_ticket_with_id.csv"

df = pd.read_csv(path)

df=df.iloc[:1]
df.to_csv('updated_bricare_1_ticket.csv', index=False)


In [31]:
import pandas as pd

# Load the data files
extract_df = pd.read_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\extract_dummyfromserver_120k.csv.xls")
bricare_df = pd.read_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\bricare_27col_100k.csv")

# Check the lengths of the CaseNumber and Ticket_ID columns
extract_length = len(extract_df['CaseNumber'])
bricare_length = len(bricare_df['Ticket_ID'])

# Ensure the number of CaseNumber matches the number of Ticket_ID
if extract_length >= bricare_length:
    # Trim the extract dataframe to match the length of the bricare dataframe
    trimmed_extract_df = extract_df.iloc[:bricare_length]
    
    # Replace Ticket_ID values with CaseNumber values
    bricare_df['Ticket_ID'] = trimmed_extract_df['CaseNumber'].values
    
    # Add the Id column from the extract dataframe to the bricare dataframe
    bricare_df['Id'] = trimmed_extract_df['Id'].values
    
    # Reorder the columns to place Id as the first column
    cols = bricare_df.columns.tolist()
    cols = ['Id'] + [col for col in cols if col != 'Id']
    bricare_df = bricare_df[cols]
    
    # Save the updated bricare dataframe to a new CSV file
    output_file_path = 'updated_bricare_120k_ticket_with_id.csv'
    bricare_df['Nomor_Kartu']='1706269727199000'
    bricare_df['No_Rekening']='878667067646102'
    
    bricare_df.to_csv(output_file_path, index=False)
    
    print("Ticket_ID values have been successfully replaced and the Id column has been added and saved to 'updated_bricare_120k_ticket_with_id.csv'.")
else:
    print("Error: The number of CaseNumber entries is less than the number of Ticket_ID entries. Ensure they are one-to-one.")

# Display the first few rows of the updated bricare dataframe
bricare_df.head()


Ticket_ID values have been successfully replaced and the Id column has been added and saved to 'updated_bricare_120k_ticket_with_id.csv'.


,Id,cifno,Ticket_ID,Call_Type_ID,Call_Type,Create_Date,gateway,Jenis_Laporan,Nama_Nasabah,No_Rekening,...,email,full_name,no_telepon,approver_login,approver_name,SLAResolution,submitter_login_id,submitter_user_group,user_login_name,record_type
0,500MR000004Q2wkYAC,LD86852,TICKET000998035,8412,Nasabah BRI Gagal Transaksi Belanja di EDC BRI...,2019-08-06 16:52:14,Phone,Complaint - Transaction,b2714defcfef8c65d3154367beb604aa,878667067646102,...,NaN,d41d8cd98f00b204e9800998ecf8427e,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,20.0,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,Case Migration
1,500MR000004Q2wlYAC,RCAW297,TICKET000998036,8412,Nasabah BRI gagal tarik tunai & terdebet di AT...,2019-08-06 16:52:14,Phone,Complaint - Transaction,b69e3868bc8b6c3231df7d691a5595fe,878667067646102,...,NaN,d41d8cd98f00b204e9800998ecf8427e,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,10.0,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,Case Migration
2,500MR000004Q2wmYAC,AOY4038,TICKET000998037,8412,Nasabah BRI Gagal Transaksi Belanja di EDC BRI...,2019-08-06 16:52:14,Phone,Complaint - Transaction,948421d448fb51f1cf27584b1256a120,878667067646102,...,NaN,d41d8cd98f00b204e9800998ecf8427e,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,20.0,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,Case Migration
3,500MR000004Q2wnYAC,AUA0263,TICKET000998038,8412,Nasabah BRI gagal transfer & terdebet jaringan...,2019-08-06 16:52:14,Phone,Complaint - Transaction,a4aa532c65d1503dcc41ef396d3c42b8,878667067646102,...,NaN,d41d8cd98f00b204e9800998ecf8427e,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,20.0,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,Case Migration
4,500MR000004Q2woYAC,HOL2556,TICKET000998039,8412,Kartu ATM BRI Tertelan di MESIN ATM,2019-08-06 16:52:14,Phone,Request,95d4f5421b5b387726b5d619dbe24155,878667067646102,...,NaN,fcc2dc136c7c3d69fb080cdd2b1648f1,NaN,NaN,d41d8cd98f00b204e9800998ecf8427e,20.0,90119221.0,LCC-CCTCALL,fcc2dc136c7c3d69fb080cdd2b1648f1,Case Migration


In [44]:
from datetime import datetime
import pytz

# Original date and time in UTC
utc_time_str_2 = "2024-06-26T21:30:20.000+0000"
utc_time_2 = datetime.strptime(utc_time_str_2, "%Y-%m-%dT%H:%M:%S.%f%z")

# Convert to Jakarta time
jakarta_tz = pytz.timezone("Asia/Jakarta")
jakarta_time_2 = utc_time_2.astimezone(jakarta_tz)

# Format the Jakarta time as a string
jakarta_time_str_2 = jakarta_time_2.strftime("%Y-%m-%d %H:%M:%S")

jakarta_time_str_2


'2024-06-27 04:30:20'

In [69]:
from datetime import datetime
import pytz

# Jakarta date and time string
jakarta_time_str_2 = "2024-06-27 01:10:20"

# Define Jakarta timezone
jakarta_tz = pytz.timezone("Asia/Jakarta")

# Parse the Jakarta time string into a datetime object
jakarta_time_2 = datetime.strptime(jakarta_time_str_2, "%Y-%m-%d %H:%M:%S")

# Localize the Jakarta time
jakarta_time_2 = jakarta_tz.localize(jakarta_time_2)

# Convert to UTC
utc_time_2 = jakarta_time_2.astimezone(pytz.utc)

# Format the UTC time as a string
utc_time_str_2 = utc_time_2.strftime("%Y-%m-%dT%H:%M:%S.%f%z")

utc_time_str_2


'2024-06-26T18:10:20.000000+0000'

In [64]:
# Reload the bricare files and the 220k file
file_220k = pd.read_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\220K.csv.xls")
bricare_1 = pd.read_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\bricare_27col_100k.csv")
bricare_2 = pd.read_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\bricare_27col_100k_2.csv")

# Merge Id and CaseNumber into bricare files
bricare_1 = bricare_1.merge(file_220k[['Id', 'CaseNumber']], left_index=True, right_index=True)
bricare_2 = bricare_2.merge(file_220k[['Id', 'CaseNumber']], left_index=True, right_index=True)

# Replace Ticket_ID with CaseNumber from 220K file
bricare_1['Ticket_ID'] = bricare_1['CaseNumber']
bricare_2['Ticket_ID'] = bricare_2['CaseNumber']

# Drop the temporary 'CaseNumber' column
bricare_1.drop(columns=['CaseNumber'], inplace=True)
bricare_2.drop(columns=['CaseNumber'], inplace=True)

# Combine the Ticket_IDs from both files to identify overlaps
combined_ticket_ids = pd.concat([bricare_1['Ticket_ID'], bricare_2['Ticket_ID']])

# Identify duplicate Ticket_IDs
duplicates = combined_ticket_ids[combined_ticket_ids.duplicated()]

# Reassign duplicates in bricare_2 to ensure uniqueness
for i, dup in enumerate(duplicates):
    bricare_2.loc[bricare_2['Ticket_ID'] == dup, 'Ticket_ID'] += f'_DUP{i}'

# Save the transformed files
bricare_1.to_csv('bricare_27col_100k_transformed_no_suffix.csv', index=False)
bricare_2.to_csv('bricare_27col_100k_2_transformed_no_suffix.csv', index=False)

# Check if Ticket_IDs are completely different between the two files
ticket_id_1 = set(bricare_1['Ticket_ID'])
ticket_id_2 = set(bricare_2['Ticket_ID'])

# Ensure no Ticket_ID is shared between the two files
unique_across_files = ticket_id_1.isdisjoint(ticket_id_2)


unique_across_files

True

In [65]:
import pandas as pd

# Load the files into pandas DataFrames
file1 = pd.read_csv(r"D:\dataquality2\bricare_27col_100k_transformed_no_suffix.csv")
file2 = pd.read_csv(r"D:\dataquality2\bricare_27col_100k_2_transformed_no_suffix.csv")

# Extract the Ticket_ID column from both DataFrames
ticket_ids_file1 = file1['Ticket_ID']
ticket_ids_file2 = file2['Ticket_ID']

# Check if there are any common values
common_ids = set(ticket_ids_file1).intersection(set(ticket_ids_file2))

if len(common_ids) == 0:
    print("All values in the Ticket_ID column are completely different between the two files.")
else:
    print("There are common values in the Ticket_ID column between the two files.")
    print("Common values:", common_ids)





All values in the Ticket_ID column are completely different between the two files.


## Compare the Group Name between SF and BRicare

from pak Suyanto
"C:\Users\maste\Downloads\dataloader_v60.0.2\server\Group name ds_ej.csv"

from SF
"C:\Users\maste\Downloads\dataloader_v60.0.2\server\CSV 2 - Group.csv"

In [76]:
import pandas as pd
import csv

import pandas as pd


file1_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\ds_ej.csv"
file2_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\CSV 2 - Group.csv"

df1 = pd.read_csv(file1_path)
df2 = pd.read_csv(file2_path)


user_groups_ds_ej = df1['usergrouptxt'].str.strip()
queue_group_names = df2['Queue & Group Name'].str.strip()


missing_groups = user_groups_ds_ej[~user_groups_ds_ej.isin(queue_group_names)]

missing_groups_list = missing_groups.tolist()
len(missing_groups_list)

996

In [6]:
# Load the CSV and Excel files
csv_file_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\Call Type Pak Hadi Green.xlsx"
excel_file_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\Call Type Cleaned.xlsx"

# Read the files
df_csv = pd.read_excel(csv_file_path)
df_excel = pd.read_excel(excel_file_path)

# Extract the relevant columns for comparison
case_types = set(df_csv['Case Types'].astype(str))
call_type_numbers = set(df_excel['Call Type Number'].astype(str))

# Find the missing and common case types
missing_case_types = case_types - call_type_numbers
common_case_types = case_types & call_type_numbers

# Save the results to CSV files
missing_case_types_df = pd.DataFrame(missing_case_types, columns=["Missing Case Types"])
common_case_types_df = pd.DataFrame(common_case_types, columns=["Common Case Types"])

missing_case_types_df.to_csv('case_types_missing.csv', index=False)
common_case_types_df.to_csv('case_types_common.csv', index=False)

# Provide the links to download the files
len(missing_case_types_df), len(common_case_types_df)


(10, 345)

In [23]:
import pandas as pd

# path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\Call Type Cleaned - TBM.xlsx"

path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\Call Type Cleaned - TBM.xlsx"


df= pd.read_excel(path)
pd.set_option('display.max_colwidth', None)
# df = df[df['Call Type Number']== 'UMI-3'] # Modify this
df['Send to Drone?'] = df['Send to Drone?'].replace({'No': 'FALSE', 'Yes': 'TRUE'})
df['Active'] = df['Active'].astype(str).str.upper()
df['external_id']=df['Call Type Number']


# df.to_csv('CallType_umi3.csv', index=False)
df.to_csv('CallType_cleaned.csv', index=False)
df

,Call Type Number,Active,Send to Drone?,Additional Details (Formulir Detail),Customer Segment,Product,Sub Product,Case Category,Case Type Description,Sub-Description,...,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,external_id
0,1000,TRUE,FALSE,nomor ponsel nasabah yang bisa dihubungi: \nID CERIA:\nNO HP yg terdaftar:\nAlasan:,Individu Umum,Loans,KTA - Digital,Inquiry,Nasabah Menanyakan Informasi Pengajuan Terkait CERIA,"1. Cara, Syarat & Ketentuan\n2. Status\n3. Pembatalan",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000
1,1002,TRUE,FALSE,NaN,Individu Umum,Loans,KTA - Digital,Products / Promotion Inquiry,Nasabah Menyakan Terkait Promo dan Program CERIA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1002
2,1003,TRUE,FALSE,NaN,Individu Umum,Loans,KTA - Digital,Request,Nasabah Mengajukan Pelunasan Awal Cicilan CERIA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1003
3,1005,TRUE,FALSE,Nasabah mengajukan pemblokiran Sementara Akun Ceria\nNomor ID :\nAlasan pemblokiran :\n\nMohon bantuan tindak lanjut\nTerima kasih,Individu Umum,Loans,KTA - Digital,Request,Nasabah Mengajukan Pemblokiran CERIA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1005
4,1008,TRUE,FALSE,NaN,Individu Umum,Loans,KTA - Digital,Request,Nasabah Mengajukan Pengaktifan Akun Ceria Terblokir karena Fraud Detection System,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,TRB-8,TRUE,FALSE,NaN,Korporasi,Servicing,Cash Management,Inquiry,Nasabah Menanyakan Pengajuan Pendaftaran Product BRIVA tidak disetujui pada QLOLA Apps,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TRB-8
380,TRB-9,TRUE,FALSE,NaN,Korporasi,Servicing,Cash Management,Inquiry,Nasabah Menanyakan Pengajuan Pendaftaran Product Corporate Billing Management (BRI CBM) tidak disetujui pada QLOLA Apps,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TRB-9
381,UMI-1,TRUE,FALSE,NaN,UMKM,Transaction Banking,Merchant Solutions,Inquiry,"Calon Mitra UMI Menanyakan Cara, Syarat, dan Ketentuan mengenai Pengajuan Menjadi Mitra UMI",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UMI-1
382,UMI-2,TRUE,FALSE,NaN,UMKM,Transaction Banking,Merchant Solutions,Inquiry,Mitra UMI Menanyakan Informasi Fee Transaksi,1. Disbursment\n2. Loan quality fee,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UMI-2


# Casenumber

In [6]:
import pandas as pd

def check_ticket_ids(file_path, case_column, output_path):
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Extract the numeric part of the ticket IDs
    df['Numeric_ID'] = df[case_column].str.extract(r'(\d+)')
    
    # Drop rows with NaN values in 'Numeric_ID'
    df = df.dropna(subset=['Numeric_ID'])
    
    # Convert the numeric part to integers
    df['Numeric_ID'] = df['Numeric_ID'].astype(int)
    
    # Sort the DataFrame by the numeric ID
    df_sorted = df.sort_values(by='Numeric_ID').reset_index(drop=True)
    
    # Find duplicates
    duplicates = df_sorted[df_sorted.duplicated(subset='Numeric_ID', keep=False)]
    
    # Find gaps
    df_sorted['Next_ID'] = df_sorted['Numeric_ID'].shift(-1)
    df_sorted['Gap'] = df_sorted['Next_ID'] - df_sorted['Numeric_ID']
    gaps = df_sorted[df_sorted['Gap'] != 1]
    
    # Detailed gap analysis
    gap_details = []
    for idx, row in gaps.iterrows():
        if not pd.isna(row['Next_ID']):
            gap_start = row[case_column]
            gap_end = df_sorted.iloc[idx + 1][case_column]
            gap_size = row['Gap'] - 1
            gap_details.append({'Gap_Start': gap_start, 'Gap_End': gap_end, 'Gap_Size': gap_size})
    
    # Save results to CSV files
    if not duplicates.empty:
        duplicates.to_csv(output_path + '_duplicates.csv', index=False)
        print("Duplicates found and saved to CSV.")
    else:
        print("No duplicates found.")
    
    if gap_details:
        gaps_df = pd.DataFrame(gap_details)
        gaps_df.to_csv(output_path + '_gaps.csv', index=False)
        print("Gaps found and saved to CSV.")
    else:
        print("No gaps found.")

# Example usage
file_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\Case_2024-07-03_okta.csv"
case_column = 'CASENUMBER'
output_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\output"

check_ticket_ids(file_path, case_column, output_path)
#select SCC_Legacy_Ticket_ID__c, CaseNumber from case where SCC_Legacy_Ticket_ID__c ='TTB0000088100' 


Duplicates found and saved to CSV.
Gaps found and saved to CSV.


In [1]:
# To spplit the extracted files, the file containing all Legacy ID having non-empty values


import os
import pandas as pd

def split_csv(input_file, output_directory, output_prefix, rows_per_file=200000):
    # Read the CSV file with semicolon as the delimiter
    df = pd.read_csv(input_file, delimiter=';')

    # Ensure the SCC_LEGACY_TICKET_ID__C column is empty
    df['SCC_LEGACY_TICKET_ID__C'] = ''

    # Get the total number of rows in the DataFrame
    total_rows = len(df)

    # Calculate the number of files needed
    num_files = (total_rows // rows_per_file) + (1 if total_rows % rows_per_file != 0 else 0)

    # Ensure the output directory exists
    os.makedirs(output_directory, exist_ok=True)

    for i in range(num_files):
        start_row = i * rows_per_file
        end_row = start_row + rows_per_file

        # Slice the DataFrame to get the chunk
        chunk = df.iloc[start_row:end_row]

        # Create a filename for the chunk
        output_file = os.path.join(output_directory, f"{output_prefix}_part_{i+1}.csv")

        # Save the chunk to a CSV file, including the header
        chunk.to_csv(output_file, index=False, sep=';')

        print(f"Saved {output_file}")

# Example usage
if __name__ == "__main__":
    input_file = r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\extract.csv"       
    output_directory = r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\file2"     
    output_prefix = 'output'
    rows_per_file = 200000                

    split_csv(input_file, output_directory, output_prefix, rows_per_file)


Saved C:\Users\maste\Downloads\dataloader_v60.0.2\output\file2\output_part_1.csv
Saved C:\Users\maste\Downloads\dataloader_v60.0.2\output\file2\output_part_2.csv
Saved C:\Users\maste\Downloads\dataloader_v60.0.2\output\file2\output_part_3.csv
Saved C:\Users\maste\Downloads\dataloader_v60.0.2\output\file2\output_part_4.csv
Saved C:\Users\maste\Downloads\dataloader_v60.0.2\output\file2\output_part_5.csv
Saved C:\Users\maste\Downloads\dataloader_v60.0.2\output\file2\output_part_6.csv
Saved C:\Users\maste\Downloads\dataloader_v60.0.2\output\file2\output_part_7.csv
Saved C:\Users\maste\Downloads\dataloader_v60.0.2\output\file2\output_part_8.csv
Saved C:\Users\maste\Downloads\dataloader_v60.0.2\output\file2\output_part_9.csv


In [3]:
import pandas as pd

path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\output\Dummy\dummy_data_casenumber60_part600.csv"

df = pd.read_csv(path)
#
df=df.iloc[:63000]
# df = df.drop(columns=['SUBJECT','PRIORITY','STATUS'])
df.to_csv(path, index=False)

df

,Status,Priority,Call Type,record_type,Legacy_ticket_id
0,Cancelled,Low,1000,Case Migration,TTB0061614201
1,Cancelled,Low,1000,Case Migration,TTB0061614202
2,Cancelled,Low,1000,Case Migration,TTB0061614203
3,Cancelled,Low,1000,Case Migration,TTB0061614204
4,Cancelled,Low,1000,Case Migration,TTB0061614205
...,...,...,...,...,...
62995,Cancelled,Low,1000,Case Migration,TTB0061677196
62996,Cancelled,Low,1000,Case Migration,TTB0061677197
62997,Cancelled,Low,1000,Case Migration,TTB0061677198
62998,Cancelled,Low,1000,Case Migration,TTB0061677199


# List of Account files in server

In [1]:
import pytesseract
from PIL import Image
import pandas as pd

# Path to your image file
image_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\WhatsApp Image 2024-07-05 at 15.51.41_53d1f445.jpg"

# Load the image using PIL
image = Image.open(image_path)

# Perform OCR on the image to extract text
extracted_text = pytesseract.image_to_string(image)

# Split the text into lines
lines = extracted_text.split('\n')

# Filter out non-CSV lines and sort them in ascending order
csv_files = sorted([line for line in lines if 'csv' in line])

# Create a DataFrame to display the sorted list of CSV files
csv_files_df = pd.DataFrame(csv_files, columns=['CSV Files'])

# Save the DataFrame to an Excel file
sorted_excel_file_path = "sorted_csv_file_names.xlsx"
csv_files_df.to_excel(sorted_excel_file_path, index=False)

print(f"Excel file with sorted CSV names saved at: {sorted_excel_file_path}")


TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.

# Update the Legacy Ticket Id into blank or NuLL

In [18]:
import pandas as pd

path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\extract_legacynonemppty77_cleaned2.csv"


df = pd.read_csv(path, delimiter=';')
df['SCC_LEGACY_TICKET_ID__C']=''
df.to_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\extract_legacynonemppty77_cleaned2.csv",index=False)
df.head()

,ID,SCC_LEGACY_TICKET_ID__C,CASENUMBER
0,500Mg000005shaOIAQ,,TTB0000122839
1,500Mg000005shaPIAQ,,TTB0000122840
2,500Mg000005shaQIAQ,,TTB0000122841
3,500Mg000005shaRIAQ,,TTB0000122842
4,500Mg000005shaSIAQ,,TTB0000122843


In [14]:
import pandas as pd

# Load the uploaded CSV file
file_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\extract_legacynonempty_1line.csv"
df = pd.read_csv(file_path, delimiter=';')

df.replace('', pd.NA, inplace=True)

df['case_origin']='X'
df['SCC_LEGACY_TICKET_ID__C']=''
df_cleaned_specific = df.dropna(subset=['ID'])

df.to_csv(r"C:\Users\maste\Downloads\dataloader_v60.0.2\cleaned_prod_1line.csv",index=False)
df_cleaned_specific.head()


,ID,SCC_LEGACY_TICKET_ID__C,CASENUMBER,case_origin
0,500Mg000005sZYzIAM,,TTB0000088100,X


# List Account

In [66]:
import re

# Path to the file
file_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\list_acount.txt"

# Regular expression pattern to match the filenames
pattern = re.compile(r'bricare_\d{8}_\d{8}_\d+_bricare_case_account_\w+\.csv')

# List to store the extracted filenames
filenames = []

# Read the file and extract filenames
with open(file_path, 'r') as file:
    for line in file:
        matches = pattern.findall(line)
        filenames.extend(matches)

# Display the extracted filenames
for filename in filenames:
    print(filename)

# Optionally, save the filenames to a new file
output_path = 'extracted_filenames.txt'
with open(output_path, 'w') as output_file:
    for filename in filenames:
        output_file.write(f"{filename}\n")

print(f"Extracted filenames saved to {output_path}")


bricare_20200806_20240430_0_bricare_case_account_business.csv
bricare_20200806_20240430_0_bricare_case_account_person_account.csv
bricare_20200806_20240430_10000001_bricare_case_account_business.csv
bricare_20200806_20240430_10000001_bricare_case_account_person_account.csv
bricare_20200806_20240430_1000001_bricare_case_account_business.csv
bricare_20200806_20240430_1000001_bricare_case_account_person_account.csv
bricare_20200806_20240430_100001_bricare_case_account_business.csv
bricare_20200806_20240430_100001_bricare_case_account_person_account.csv
bricare_20200806_20240430_10100001_bricare_case_account_business.csv
bricare_20200806_20240430_10100001_bricare_case_account_person_account.csv
bricare_20200806_20240430_10200001_bricare_case_account_business.csv
bricare_20200806_20240430_10200001_bricare_case_account_person_account.csv
bricare_20200806_20240430_10300001_bricare_case_account_business.csv
bricare_20200806_20240430_10300001_bricare_case_account_person_account.csv
bricare_2020

In [85]:
import re
import dns.resolver

def is_valid_email(email):
    regex = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    if re.match(regex, email) is None:
        return False
    
    domain = email.split('@')[1]
    try:
        dns.resolver.resolve(domain, 'MX')
    except (dns.resolver.NoAnswer, dns.resolver.NXDOMAIN, dns.resolver.Timeout):
        return False
    
    return True

def correct_email(email):
    corrections = {
        'gmial.com': 'gmail.com',
        'gamil.com': 'gmail.com',
        'yahooo.com': 'yahoo.com',
        'hotmial.com': 'hotmail.com',
        'hotmal.com': 'hotmail.com',
    }
    
    if '@' not in email:
        return email
    
    local_part, domain = email.split('@')
    
    # Common typo corrections for domains
    if domain in corrections:
        domain = corrections[domain]
    
    # Repeated characters correction (e.g., "john..doe@gmail.com")
    local_part = re.sub(r'\.{2,}', '.', local_part)
    
    corrected_email = f"{local_part}@{domain}"
    
    if is_valid_email(corrected_email):
        return corrected_email
    else:
        return email

# Example usage
emails = [
    "john..doe@gmial.com",
    "invalid-email.com",
    "jane.doe@yahooo.com",
    "test@invalid-domain.com",
    "hilmi@gmail.com master"
]

corrected_emails = [correct_email(email) for email in emails]
print("Corrected emails:", corrected_emails)


Corrected emails: ['john.doe@gmail.com', 'invalid-email.com', 'jane.doe@yahoo.com', 'test@invalid-domain.com', 'hilmi@gmail.com master']


# To Compare the Queue Name and Excalation Team

In [ ]:
# To Compare the Queue Name and Excalation Team

import pandas as pd
from difflib import get_close_matches

# Load the Excel file
excel_file_path = '/mnt/data/(FINAL) SLA&OLA_NewUserGrouping_Ringkasan (3).xlsx'
excel_data = pd.read_excel(excel_file_path, sheet_name='Appendix Team Name')

# Load the CSV file
csv_file_path = '/mnt/data/extract_Que.csv'
extract_que_df = pd.read_csv(csv_file_path)

# Extract relevant columns
appendix_names = excel_data['Full Name / Queue Name'].dropna().unique()
extract_names = extract_que_df['NAME'].dropna().unique()

# Find matches
matches = {name: get_close_matches(name, extract_names, n=1, cutoff=0.6) for name in appendix_names}

# Filter out names without matches
matches = {k: v for k, v in matches.items() if v}

# Convert matches to a DataFrame for better readability
matches_df = pd.DataFrame([(k, v[0]) for k, v in matches.items()], columns=['Appendix Team Name', 'Extract Name'])

# Define a function to describe the differences between two strings
def describe_difference(str1, str2):
    if str1 == str2:
        return "Exact match"
    differences = []
    if len(str1) != len(str2):
        differences.append("Different lengths")
    if str1.strip() != str2.strip():
        differences.append("Whitespace differences")
    for i, (char1, char2) in enumerate(zip(str1, str2)):
        if char1 != char2:
            differences.append(f"Different character at position {i}: '{char1}' vs '{char2}'")
    return "; ".join(differences) if differences else "Other differences"

# Add a column describing the differences
matches_df['Difference Details'] = matches_df.apply(lambda row: describe_difference(row['Appendix Team Name'], row['Extract Name']), axis=1)

import ace_tools as tools; tools.display_dataframe_to_user(name="Matched Names with Differences", dataframe=matches_df)

matches_df.head()


In [2]:
import pandas as pd

path=r"C:\Users\maste\Downloads\Matched_Names_with_Differences.csv"

df=pd.read_csv(path)
df.to_excel('match_queue_name_vs_escalation_name.xlsx', index=False)



# Combine all error logs into 2 different files: Business and Person Account

In [10]:
import os
import pandas as pd

folder_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\account"


all_files = os.listdir(folder_path)


error_files = [f for f in all_files if "error" in f]


business_data = pd.DataFrame()
person_account_data = pd.DataFrame()


for file_name in error_files:
    file_path = os.path.join(folder_path, file_name)
    data = pd.read_csv(file_path, delimiter=';')
    

    data['Nama'] = data['Nama'].apply(lambda x: x[-250:] if isinstance(x, str) else x)
    
    # combine the columns "Email" and "no telp"
    data['Description'] = data[['No_telp', 'Email']].apply(lambda x: ' | '.join(x.dropna().astype(str)), axis=1)
    

    data = data.drop(columns=['No_telp', 'Email'])
    

    business_data = pd.concat([business_data, data[data['record_type'] == 'Business']])
    person_account_data = pd.concat([person_account_data, data[data['record_type'] == 'Person Account']])




business_file_path = os.path.join(folder_path, 'account_business_records.csv')
person_account_file_path = os.path.join(folder_path, 'account_person_account_records.csv')


business_data.to_csv(business_file_path, index=False)
person_account_data.to_csv(person_account_file_path, index=False)

print(f"Business records saved to: {business_file_path}")
print(f"Person Account records saved to: {person_account_file_path}")

Business records saved to: C:\Users\maste\Downloads\dataloader_v60.0.2\server\account\account_business_records.csv
Person Account records saved to: C:\Users\maste\Downloads\dataloader_v60.0.2\server\account\account_person_account_records.csv


# To check and analyze the Account data

In [ ]:
import csv
import os

def detect_delimiter(file_path):
    with open(file_path, 'r') as file:
        header = file.readline()
        for delimiter in [',', ';', '\t', '|']:
            if delimiter in header:
                return delimiter
    return None

def analyze_file(file_path, expected_delimiter):
    with open(file_path, 'r') as file:
        reader = csv.reader(file, delimiter=expected_delimiter)
        line_number = 0
        inconsistent_lines = []
        for row in reader:
            line_number += 1
            if len(row) != len(reader.fieldnames):
                inconsistent_lines.append(line_number)
        return inconsistent_lines

def main():
    file_paths = ["path_to_your_first_csv_file.csv", "path_to_your_second_csv_file.csv"]  

    for file_path in file_paths:
        if not os.path.isfile(file_path):
            print(f"File not found: {file_path}")
            continue
        
        detected_delimiter = detect_delimiter(file_path)
        if detected_delimiter:
            print(f"Detected delimiter for {file_path}: {detected_delimiter}")
            inconsistent_lines = analyze_file(file_path, detected_delimiter)
            if inconsistent_lines:
                print(f"Inconsistent lines in {file_path}: {inconsistent_lines}")
            else:
                print(f"No inconsistencies found in {file_path}")
        else:
            print(f"Could not detect delimiter for {file_path}")

if __name__ == "__main__":
    main()


In [2]:
import csv
import os

def detect_delimiter(file_path):
    with open(file_path, 'r') as file:
        header = file.readline()
        for delimiter in [',', ';', '\t', '|']:
            if delimiter in header:
                return delimiter
    return None

def analyze_file(file_path, expected_delimiter):
    with open(file_path, 'r') as file:
        reader = csv.reader(file, delimiter=expected_delimiter)
        line_number = 0
        inconsistent_lines = []
        field_count = len(next(reader))
        for row in reader:
            line_number += 1
            if len(row) != field_count:
                inconsistent_lines.append(line_number)
        return inconsistent_lines

def process_folder(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".csv"):
                file_path = os.path.join(root, file)
                print(f"Processing file: {file_path}")

                detected_delimiter = detect_delimiter(file_path)
                if detected_delimiter:
                    print(f"Detected delimiter for {file_path}: {detected_delimiter}")
                    inconsistent_lines = analyze_file(file_path, detected_delimiter)
                    if inconsistent_lines:
                        print(f"Inconsistent lines in {file_path}: {inconsistent_lines}")
                    else:
                        print(f"No inconsistencies found in {file_path}")
                else:
                    print(f"Could not detect delimiter for {file_path}")

def main():
    folder_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\test"
    if os.path.isdir(folder_path):
        process_folder(folder_path)
    else:
        print(f"Folder not found: {folder_path}")

if __name__ == "__main__":
    main()


Processing file: C:\Users\maste\Downloads\dataloader_v60.0.2\server\test\error071124083558030.csv
Detected delimiter for C:\Users\maste\Downloads\dataloader_v60.0.2\server\test\error071124083558030.csv: ;
Inconsistent lines in C:\Users\maste\Downloads\dataloader_v60.0.2\server\test\error071124083558030.csv: [1]
Processing file: C:\Users\maste\Downloads\dataloader_v60.0.2\server\test\error071124083558031.csv
Detected delimiter for C:\Users\maste\Downloads\dataloader_v60.0.2\server\test\error071124083558031.csv: ;
Inconsistent lines in C:\Users\maste\Downloads\dataloader_v60.0.2\server\test\error071124083558031.csv: [2, 3]


In [4]:
import pandas as pd




import csv

# Input and output file paths
input_file = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\account\error071124083558031.csv"
output_file = r"c:\Users\maste\Downloads\dataloader_v60.0.2\server\account\error071124083558030_output.csv"

# Read the input CSV file
with open(input_file, 'r', newline='', encoding='utf-8') as infile:
    reader = csv.reader(infile, delimiter=';')
    rows = list(reader)

# Process the rows to add double quotes where necessary
processed_rows = []
for row in rows:
    if row == rows[0]:
        # Keep the header row as is
        processed_rows.append(row)
    else:
        # Add double quotes around non-empty values
        processed_row = ['' if value == '""' else f'"{value}"' if value else '' for value in row]
        processed_rows.append(processed_row)

# Write the processed rows to the output CSV file
with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
    writer = csv.writer(outfile, delimiter=';')
    writer.writerows(processed_rows)

print("CSV file has been processed and saved.")


CSV file has been processed and saved.


In [6]:
import os
import pandas as pd

def save_shapes_to_excel(folder_path, output_file):
    data = []


    for filename in os.listdir(folder_path):

        if filename.endswith('.csv'):
     
            file_path = os.path.join(folder_path, filename)
      
            df = pd.read_csv(file_path)
    
            rows, columns = df.shape
         
            data.append([filename, rows, columns])

 
    df_shapes = pd.DataFrame(data, columns=['Name', 'Rows', 'Columns'])


    df_shapes.to_excel(output_file, index=False)

    print(f"Shapes of CSV files saved to {output_file}")


folder_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\server\account"
output_file = 'csv_shapes.xlsx'


save_shapes_to_excel(folder_path, output_file)


Shapes of CSV files saved to csv_shapes.xlsx


# Role 

In [5]:
import pandas as pd
import re

# Load the Excel file
file_path = r"C:\Users\maste\Downloads\BRI - User Matrix v1.2.xlsx"
df = pd.read_excel(file_path, sheet_name='Role2')

df['']

df.to_csv('Role2.csv',index=False)

In [8]:
import pandas as pd

path=r"D:\dataquality2\bricare_uat20230101_20230101.csv"

df= pd.read_csv(path)

df['Chanel'].unique()

array([nan, 'ATM BANK LAIN', 'Aplikasi', 'EDC', 'CRM', 'ATM', 'ATM BRI'],
      dtype=object)

In [31]:
import os
import pandas as pd

def format_ticket_id(ticket_id):
    if pd.notna(ticket_id) and ticket_id.startswith("TTB"):
        number_part = ticket_id[3:]
        formatted_number = number_part[-10:].zfill(10)
        return f"TTB{formatted_number}"
    return ticket_id

def process_file(file_path):
    try:
        df = pd.read_csv(file_path)
        if "Ticket_ID" in df.columns:
            df["Ticket_ID"] = df["Ticket_ID"].apply(format_ticket_id)
        if "Legacy_Ticket_ID" in df.columns:
            df["Legacy_Ticket_ID"] = df["Legacy_Ticket_ID"].apply(format_ticket_id)
        output_path = file_path.replace(".csv", "_processed.csv")
        df.to_csv(output_path, index=False)
        print(f"Processed and saved: {output_path}")
        return df
    except PermissionError as e:
        print(f"PermissionError: {e}")
    except FileNotFoundError as e:
        print(f"FileNotFoundError: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

def process_all_files_in_folder(folder_path):
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".csv"):
            file_path = os.path.join(folder_path, file_name)
            process_file(file_path)


folder_path = r"D:\test_case - Copy"
process_all_files_in_folder(folder_path)


Processed and saved: D:\test_case - Copy\dummy_data_case_uat - 27 - Copy_processed.csv
Processed and saved: D:\test_case - Copy\dummy_data_case_uat - 27 - Copy_processed_processed.csv
Processed and saved: D:\test_case - Copy\dummy_data_case_uat - 79 - Copy_processed.csv
Processed and saved: D:\test_case - Copy\dummy_data_case_uat - 79 - Copy_processed_processed.csv
Processed and saved: D:\test_case - Copy\dummy_data_case_uat - 79 - Copy_processed_processed_processed.csv


In [32]:
import os
import csv

def format_ticket_id(ticket_id):
    if ticket_id and ticket_id.startswith("TTB"):
        number_part = ticket_id[3:]
        formatted_number = number_part[-10:].zfill(10)
        return f"TTB{formatted_number}"
    return ticket_id

def process_file(file_path):
    try:
        with open(file_path, 'r', newline='', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            rows = list(reader)
            fieldnames = reader.fieldnames
        

        for row in rows:
            if "Ticket_ID" in row:
                row["Ticket_ID"] = format_ticket_id(row["Ticket_ID"])
            if "Legacy_Ticket_ID" in row:
                row["Legacy_Ticket_ID"] = format_ticket_id(row["Legacy_Ticket_ID"])
        

        output_path = file_path.replace(".csv", "_processed.csv")
        with open(output_path, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(rows)
        
        print(f"Processed and saved: {output_path}")
    except PermissionError as e:
        print(f"PermissionError: {e}")
    except FileNotFoundError as e:
        print(f"FileNotFoundError: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

def process_all_files_in_folder(folder_path):
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".csv"):
            file_path = os.path.join(folder_path, file_name)
            process_file(file_path)


folder_path = r"D:\test_case - Copy"
process_all_files_in_folder(folder_path)



Processed and saved: D:\test_case - Copy\dummy_data_case_uat - 27 - Copy_processed.csv
Processed and saved: D:\test_case - Copy\dummy_data_case_uat - 27 - Copy_processed_processed.csv
Processed and saved: D:\test_case - Copy\dummy_data_case_uat - 27 - Copy_processed_processed_processed.csv
Processed and saved: D:\test_case - Copy\dummy_data_case_uat - 79 - Copy_processed.csv
Processed and saved: D:\test_case - Copy\dummy_data_case_uat - 79 - Copy_processed_processed.csv
Processed and saved: D:\test_case - Copy\dummy_data_case_uat - 79 - Copy_processed_processed_processed.csv
Processed and saved: D:\test_case - Copy\dummy_data_case_uat - 79 - Copy_processed_processed_processed_processed.csv


In [26]:
import pandas as pd

path=r"D:\test_case - Copy\dummy_data_case_uat - 79 - Copy.csv"

df=pd.read_csv(path)
df

,case_number,record_type,cif,Ticket_ID,Call_Type_ID,Call_Type,Details,Create_Date,gateway,Jenis_Laporan,...,Tanggal_Settlement,Tgl_Foward,Tgl_In_Progress,Tgl_Returned,Ticket_Referensi,Tiket_Urgency,Tipe_Remark,UniqueID,users,Usergroup_ID
0,TICKET001339022,Case Migration,STGD 980,TTB001339022,1000,Blokir Kartu ATM karena kartu hilang,Nasabah mengajukan pemblokiran kartu ATM BRI\n...,2023-01-01 07:07:15,Phone,Request,...,NaN,NaN,NaN,NaN,NaN,NaN,Notes,NaN,Call,4
